In [ ]:
cd /home/monierashraf/Desktop/llm/Row_match_recognize && python3 trino_test_replication.py

# Example 1

In [1]:
import pandas as pd
from src.executor.match_recognize import match_recognize
import pandas as pd

# Define the data
data = [
    ('cust_1', '2020-05-11', 100),
    ('cust_1', '2020-05-12', 200),
    ('cust_2', '2020-05-13',   8),
    ('cust_1', '2020-05-14', 100),
    ('cust_2', '2020-05-15',   4),
    ('cust_1', '2020-05-16',  50),
    ('cust_1', '2020-05-17', 100),
    ('cust_2', '2020-05-18',   6),
]

# Create DataFrame
df = pd.DataFrame(data, columns=['customer_id', 'order_date', 'price'])

# Convert order_date column to datetime
df['order_date'] = pd.to_datetime(df['order_date'])

# Display the DataFrame
print(df)

query_basic = """
SELECT customer_id, start_price, bottom_price, final_price, start_date, final_date
FROM memory.default.orders
MATCH_RECOGNIZE (
PARTITION BY customer_id
ORDER BY order_date
MEASURES
START.price AS start_price,
LAST(DOWN.price) AS bottom_price,
LAST(UP.price) AS final_price,
START.order_date AS start_date,
LAST(UP.order_date) AS final_date
ONE ROW PER MATCH
AFTER MATCH SKIP PAST LAST ROW
PATTERN (START DOWN+ UP+)
DEFINE
DOWN AS price < PREV(price),
UP AS price > PREV(price)
);

"""

print("Test 1:")
output_df = match_recognize(query_basic, df)
print(output_df)
print("\n")

INFO:row_match_recognize.src.matcher.production_aggregates:MeasureEvaluator enhanced with production aggregate support
INFO:row_match_recognize.src.executor.match_recognize:Production aggregates enabled for MeasureEvaluator
DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT customer_id, start_price, bottom_price, final_price, start_date, final_date FROM memory.default.orders MATCH_RECOGNIZE ( PARTITION BY customer_id ORDER BY order_date MEASURES START.price AS start_price, LAST(DOWN.price) AS bottom_price, LAST(UP.price) AS final_price, START.order_date AS start_date, LAST(UP.order_date) AS final_date ONE ROW PER MATCH AFTER MATCH SKIP PAST LAST ROW PATTERN (START DOWN+ UP+) DEFINE DOWN AS price < PREV(price), UP AS price > PREV(price) );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=customer_id, metadata={}), SelectItem(expression=start_price, metadata={}), SelectItem(expression=bottom_price, metadata=

  customer_id order_date  price
0      cust_1 2020-05-11    100
1      cust_1 2020-05-12    200
2      cust_2 2020-05-13      8
3      cust_1 2020-05-14    100
4      cust_2 2020-05-15      4
5      cust_1 2020-05-16     50
6      cust_1 2020-05-17    100
7      cust_2 2020-05-18      6
Test 1:
Pattern value: 'START DOWN+ UP+'
Pattern variables {'START'} have no DEFINE conditions - defaulting to TRUE (always match)
Pattern value: 'START DOWN+ UP+'
Pattern variables {'START'} have no DEFINE conditions - defaulting to TRUE (always match)
DEBUG: DFA metadata keys: ['pattern_variables', 'variables_with_quantifiers', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBU

INFO:row_match_recognize.src.matcher.production_aggregates:ENHANCED_EVAL_DEBUG: Original evaluator returned: 50 (type: <class 'int'>)
DEBUG:row_match_recognize.src.matcher.matcher:Setting bottom_price to 50 from evaluator
DEBUG:row_match_recognize.src.matcher.production_aggregates:Enhanced evaluate called with expr: 'LAST(UP.price)', semantics: 'FINAL'
DEBUG:row_match_recognize.src.matcher.production_aggregates:Pattern match result for 'LAST(UP.price)': <re.Match object; span=(0, 5), match='LAST('>
DEBUG:row_match_recognize.src.matcher.production_aggregates:Matched function: LAST, semantics_prefix: None
DEBUG:row_match_recognize.src.matcher.production_aggregates:Function LAST not in standard aggregates, using original evaluator
INFO:row_match_recognize.src.matcher.production_aggregates:ENHANCED_EVAL_DEBUG: Falling back to original evaluator for: LAST(UP.price)
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Evaluating expression: LAST(UP.price) with FINAL semantics
DEBUG:row_ma

DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: False
DEBUG: has_complex_back_references: False
DEBUG: Processing row 0 with value N/A
DEBUG: Found 1 valid transitions: ['START']
DEBUG: Categorized transitions: {'accepting': [], 'prerequisite': [], 'simple': [('START', 1, False)], 'dependent': []}
DEBUG: Processing category 'simple' with 1 transitions
DEBUG: Selected simple transition: START -> state 1 (alternation priority: N/A)
DEBUG: Processing row 1 with value N/A
DEBUG: Found 1 valid transitions: ['DOWN']
DEBUG: Categorized transitions: {'accepting': [], 'prerequisite': [], 'simple': [('DOWN', 2, False)], 'dependent': []}
DEBUG: Processing category 'simple' with 1 transitions
DEBUG: Selected simple transition: DOWN -> state 2 (alternation priority: N/A)
DEBUG: Processing row 2 with value N/A
DEBUG: Found 1 valid transitions: ['UP']
DEBUG: Categorized transitions: {'accepting': [('UP', 3, False)], 'prereq

# check_fix

In [2]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Simple test case to debug PREV function issue
data = [
    ('cust_1', '2020-05-11', 100),
    ('cust_1', '2020-05-12', 200),
    ('cust_1', '2020-05-14', 100),
    ('cust_1', '2020-05-16', 50),
    ('cust_1', '2020-05-17', 100),
]

# Create DataFrame
df = pd.DataFrame(data, columns=['customer_id', 'order_date', 'price'])
df['order_date'] = pd.to_datetime(df['order_date'])

print("Test data:")
print(df)
print()

# Test simple query without PREV first
query_simple = """
SELECT customer_id, start_price, final_price, start_date, final_date
    FROM memory.default.orders
        MATCH_RECOGNIZE (
            PARTITION BY customer_id
            ORDER BY order_date
            MEASURES
                START.price AS start_price,
                LAST(DOWN.price) AS final_price,
                START.order_date AS start_date,
                LAST(DOWN.order_date) AS final_date
            ONE ROW PER MATCH
            AFTER MATCH SKIP PAST LAST ROW
            PATTERN (START DOWN+)
            DEFINE
                DOWN AS price < 150
            );
"""

print("Testing simple query without PREV:")
try:
    result = match_recognize(query_simple, df)
    print(result)
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()
print()

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT customer_id, start_price, final_price, start_date, final_date FROM memory.default.orders MATCH_RECOGNIZE ( PARTITION BY customer_id ORDER BY order_date MEASURES START.price AS start_price, LAST(DOWN.price) AS final_price, START.order_date AS start_date, LAST(DOWN.order_date) AS final_date ONE ROW PER MATCH AFTER MATCH SKIP PAST LAST ROW PATTERN (START DOWN+) DEFINE DOWN AS price < 150 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=customer_id, metadata={}), SelectItem(expression=start_price, metadata={}), SelectItem(expression=final_price, metadata={}), SelectItem(expression=start_date, metadata={}), SelectItem(expression=final_date, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_e

Test data:
  customer_id order_date  price
0      cust_1 2020-05-11    100
1      cust_1 2020-05-12    200
2      cust_1 2020-05-14    100
3      cust_1 2020-05-16     50
4      cust_1 2020-05-17    100

Testing simple query without PREV:
Pattern value: 'START DOWN+'
Pattern variables {'START'} have no DEFINE conditions - defaulting to TRUE (always match)
Pattern value: 'START DOWN+'
Pattern variables {'START'} have no DEFINE conditions - defaulting to TRUE (always match)
DEBUG: DFA metadata keys: ['pattern_variables', 'variables_with_quantifiers', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: False
DEBUG: has_alternations: False
DEBUG: No al

In [4]:
import pandas as pd
from src.executor.match_recognize import match_recognize
import pandas as pd

# Define the data
data = [
    ('cust_1', '2020-05-11', 100),
    ('cust_1', '2020-05-12', 200),
    ('cust_2', '2020-05-13',   8),
    ('cust_1', '2020-05-14', 100),
    ('cust_2', '2020-05-15',   4),
    ('cust_1', '2020-05-16',  50),
    ('cust_1', '2020-05-17', 100),
    ('cust_2', '2020-05-18',   6),
]

# Create DataFrame
df = pd.DataFrame(data, columns=['customer_id', 'order_date', 'price'])

# Convert order_date column to datetime
df['order_date'] = pd.to_datetime(df['order_date'])

# Display the DataFrame
print(df)

query_basic = """
SELECT customer_id, start_date, end_date, bottom_price
FROM memory.default.orders
MATCH_RECOGNIZE (
PARTITION BY customer_id
ORDER BY order_date
MEASURES
A.order_date AS start_date,
C.order_date AS end_date,
C.price AS bottom_price
PATTERN (A B C)
DEFINE
B AS B.price < A.price,
C AS C.price < B.price
);


"""

print("Test 1:")
output_df = match_recognize(query_basic, df)
print(output_df)
print("\n")

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT customer_id, start_date, end_date, bottom_price FROM memory.default.orders MATCH_RECOGNIZE ( PARTITION BY customer_id ORDER BY order_date MEASURES A.order_date AS start_date, C.order_date AS end_date, C.price AS bottom_price PATTERN (A B C) DEFINE B AS B.price < A.price, C AS C.price < B.price );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=customer_id, metadata={}), SelectItem(expression=start_date, metadata={}), SelectItem(expression=end_date, metadata={}), SelectItem(expression=bottom_price, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['customer_id'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: Orde

  customer_id order_date  price
0      cust_1 2020-05-11    100
1      cust_1 2020-05-12    200
2      cust_2 2020-05-13      8
3      cust_1 2020-05-14    100
4      cust_2 2020-05-15      4
5      cust_1 2020-05-16     50
6      cust_1 2020-05-17    100
7      cust_2 2020-05-18      6
Test 1:
Pattern value: 'A B C'
Pattern variables {'A'} have no DEFINE conditions - defaulting to TRUE (always match)
Pattern value: 'A B C'
Pattern variables {'A'} have no DEFINE conditions - defaulting to TRUE (always match)
DEBUG: DFA metadata keys: ['pattern_variables', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: False
DEBUG: has_alternations: False
DEBUG

In [5]:
import pandas as pd
from src.executor.match_recognize import match_recognize
import pandas as pd

# Define the data
data = [
    ('cust_1', '2020-05-11', 100),
    ('cust_1', '2020-05-12', 200),
    ('cust_2', '2020-05-13',   8),
    ('cust_1', '2020-05-14', 100),
    ('cust_2', '2020-05-15',   4),
    ('cust_1', '2020-05-16',  50),
    ('cust_1', '2020-05-17', 100),
    ('cust_2', '2020-05-18',   6),
]

# Create DataFrame
df = pd.DataFrame(data, columns=['customer_id', 'order_date', 'price'])

# Convert order_date column to datetime
df['order_date'] = pd.to_datetime(df['order_date'])

# Display the DataFrame
print(df)

query_basic = """
SELECT
customer_id,
start_price,
last_b_price AS peak_price,
last_c_price AS bottom_price,
last_d_price AS end_price,
start_date,
end_date
FROM memory.default.orders
MATCH_RECOGNIZE (
PARTITION BY customer_id
ORDER BY order_date
MEASURES
A.price AS start_price,
LAST(B.price) AS last_b_price,
LAST(C.price) AS last_c_price,
LAST(D.price) AS last_d_price,
A.order_date AS start_date,
LAST(D.order_date) AS end_date
PATTERN (A B+ C+ D+)
DEFINE
B AS B.price > PREV(price),
C AS C.price < PREV(price),
D AS D.price > PREV(price)
);

"""

print("Test 1:")
output_df = match_recognize(query_basic, df)
print(output_df)
print("\n")

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT customer_id, start_price, last_b_price AS peak_price, last_c_price AS bottom_price, last_d_price AS end_price, start_date, end_date FROM memory.default.orders MATCH_RECOGNIZE ( PARTITION BY customer_id ORDER BY order_date MEASURES A.price AS start_price, LAST(B.price) AS last_b_price, LAST(C.price) AS last_c_price, LAST(D.price) AS last_d_price, A.order_date AS start_date, LAST(D.order_date) AS end_date PATTERN (A B+ C+ D+) DEFINE B AS B.price > PREV(price), C AS C.price < PREV(price), D AS D.price > PREV(price) );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=customer_id, metadata={}), SelectItem(expression=start_price, metadata={}), SelectItem(expression=last_b_price, alias=peak_price, metadata={}), SelectItem(expression=last_c_price, alias=bottom_price, metadata={}), SelectItem(expression=last_d_price, alias=end_price, metadata={}), SelectItem(expres

  customer_id order_date  price
0      cust_1 2020-05-11    100
1      cust_1 2020-05-12    200
2      cust_2 2020-05-13      8
3      cust_1 2020-05-14    100
4      cust_2 2020-05-15      4
5      cust_1 2020-05-16     50
6      cust_1 2020-05-17    100
7      cust_2 2020-05-18      6
Test 1:
Pattern value: 'A B+ C+ D+'
Pattern variables {'A'} have no DEFINE conditions - defaulting to TRUE (always match)
Pattern value: 'A B+ C+ D+'
Pattern variables {'A'} have no DEFINE conditions - defaulting to TRUE (always match)
DEBUG: DFA metadata keys: ['pattern_variables', 'variables_with_quantifiers', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: Fa

In [6]:
import pandas as pd
from src.executor.match_recognize import match_recognize
import pandas as pd

# Define the data
data = [
    ('cust_1', '2020-05-11', 100),
    ('cust_1', '2020-05-12', 200),
    ('cust_2', '2020-05-13',   8),
    ('cust_1', '2020-05-14', 100),
    ('cust_2', '2020-05-15',   4),
    ('cust_1', '2020-05-16',  50),
    ('cust_1', '2020-05-17', 100),
    ('cust_2', '2020-05-18',   6),
]

# Create DataFrame
df = pd.DataFrame(data, columns=['customer_id', 'order_date', 'price'])

# Convert order_date column to datetime
df['order_date'] = pd.to_datetime(df['order_date'])

# Display the DataFrame
print(df)

query_basic = """
SELECT
customer_id,
a_date AS first_date,
b_date AS second_date,
a_price AS first_price,
b_price AS second_price
FROM memory.default.orders
MATCH_RECOGNIZE (
PARTITION BY customer_id
ORDER BY order_date
MEASURES
A.order_date AS a_date,
B.order_date AS b_date,
A.price AS a_price,
B.price AS b_price
PATTERN (^A B)
DEFINE
A AS price < 150,
B AS price > PREV(price)
);

"""

print("Test 1:")
output_df = match_recognize(query_basic, df)
print(output_df)
print("\n")

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT customer_id, a_date AS first_date, b_date AS second_date, a_price AS first_price, b_price AS second_price FROM memory.default.orders MATCH_RECOGNIZE ( PARTITION BY customer_id ORDER BY order_date MEASURES A.order_date AS a_date, B.order_date AS b_date, A.price AS a_price, B.price AS b_price PATTERN (^A B) DEFINE A AS price < 150, B AS price > PREV(price) );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=customer_id, metadata={}), SelectItem(expression=a_date, alias=first_date, metadata={}), SelectItem(expression=b_date, alias=second_date, metadata={}), SelectItem(expression=a_price, alias=first_price, metadata={}), SelectItem(expression=b_price, alias=second_price, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG

  customer_id order_date  price
0      cust_1 2020-05-11    100
1      cust_1 2020-05-12    200
2      cust_2 2020-05-13      8
3      cust_1 2020-05-14    100
4      cust_2 2020-05-15      4
5      cust_1 2020-05-16     50
6      cust_1 2020-05-17    100
7      cust_2 2020-05-18      6
Test 1:
Pattern value: '^A B'
Pattern value: '^A B'
DEBUG: DFA metadata keys: ['has_start_anchor', 'pattern_variables', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: False
DEBUG: has_alternations: False
DEBUG: No alternation_combinations in DFA metadata
DEBUG: Parsed alternation_order: {}
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alt

In [7]:
import pandas as pd
from src.executor.match_recognize import match_recognize
import pandas as pd

# Define the data
data = [
    ('cust_1', '2020-05-11', 100),
    ('cust_1', '2020-05-12', 200),
    ('cust_2', '2020-05-13',   8),
    ('cust_1', '2020-05-14', 100),
    ('cust_2', '2020-05-15',   4),
    ('cust_1', '2020-05-16',  50),
    ('cust_1', '2020-05-17', 100),
    ('cust_2', '2020-05-18',   6),
]

# Create DataFrame
df = pd.DataFrame(data, columns=['customer_id', 'order_date', 'price'])

# Convert order_date column to datetime
df['order_date'] = pd.to_datetime(df['order_date'])

# Display the DataFrame
print(df)

query_basic = """
SELECT
customer_id,
run_start,
run_end
FROM memory.default.orders
MATCH_RECOGNIZE (
PARTITION BY customer_id
ORDER BY order_date
MEASURES
A.order_date AS run_start,
LAST(B.order_date) AS run_end
PATTERN (A B+)
DEFINE
A AS price >= 100,
B AS price > PREV(price)
);

"""

print("Test 1:")
output_df = match_recognize(query_basic, df)
print(output_df)
print("\n")

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT customer_id, run_start, run_end FROM memory.default.orders MATCH_RECOGNIZE ( PARTITION BY customer_id ORDER BY order_date MEASURES A.order_date AS run_start, LAST(B.order_date) AS run_end PATTERN (A B+) DEFINE A AS price >= 100, B AS price > PREV(price) );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=customer_id, metadata={}), SelectItem(expression=run_start, metadata={}), SelectItem(expression=run_end, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['customer_id'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='order_date', ordering='ASC', nulls_ordering=None)])
DE

  customer_id order_date  price
0      cust_1 2020-05-11    100
1      cust_1 2020-05-12    200
2      cust_2 2020-05-13      8
3      cust_1 2020-05-14    100
4      cust_2 2020-05-15      4
5      cust_1 2020-05-16     50
6      cust_1 2020-05-17    100
7      cust_2 2020-05-18      6
Test 1:
Pattern value: 'A B+'
Pattern value: 'A B+'
DEBUG: DFA metadata keys: ['pattern_variables', 'variables_with_quantifiers', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: False
DEBUG: has_alternations: False
DEBUG: No alternation_combinations in DFA metadata
DEBUG: Parsed alternation_order: {}
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_p

In [8]:
import pandas as pd
from src.executor.match_recognize import match_recognize
import pandas as pd

# Define the data
data = [
    ('cust_1', '2020-05-11', 100),
    ('cust_1', '2020-05-12', 200),
    ('cust_2', '2020-05-13',   8),
    ('cust_1', '2020-05-14', 100),
    ('cust_2', '2020-05-15',   4),
    ('cust_1', '2020-05-16',  50),
    ('cust_1', '2020-05-17', 100),
    ('cust_2', '2020-05-18',   6),
]

# Create DataFrame
df = pd.DataFrame(data, columns=['customer_id', 'order_date', 'price'])

# Convert order_date column to datetime
df['order_date'] = pd.to_datetime(df['order_date'])

# Display the DataFrame
print(df)

query_basic = """
SELECT
customer_id,
first_a_date AS a_date,
first_b_date AS b_date,
first_c_date AS c_date
FROM memory.default.orders
MATCH_RECOGNIZE (
PARTITION BY customer_id
ORDER BY order_date
MEASURES
A.order_date AS first_a_date,
B.order_date AS first_b_date,
C.order_date AS first_c_date
PATTERN (PERMUTE(A, B, C))
DEFINE
A AS price = 100,
B AS price = 50,
C AS price = 200
);

"""

print("Test 1:")
output_df = match_recognize(query_basic, df)
print(output_df)
print("\n")

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT customer_id, first_a_date AS a_date, first_b_date AS b_date, first_c_date AS c_date FROM memory.default.orders MATCH_RECOGNIZE ( PARTITION BY customer_id ORDER BY order_date MEASURES A.order_date AS first_a_date, B.order_date AS first_b_date, C.order_date AS first_c_date PATTERN (PERMUTE(A, B, C)) DEFINE A AS price = 100, B AS price = 50, C AS price = 200 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=customer_id, metadata={}), SelectItem(expression=first_a_date, alias=a_date, metadata={}), SelectItem(expression=first_b_date, alias=b_date, metadata={}), SelectItem(expression=first_c_date, alias=c_date, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY:

  customer_id order_date  price
0      cust_1 2020-05-11    100
1      cust_1 2020-05-12    200
2      cust_2 2020-05-13      8
3      cust_1 2020-05-14    100
4      cust_2 2020-05-15      4
5      cust_1 2020-05-16     50
6      cust_1 2020-05-17    100
7      cust_2 2020-05-18      6
Test 1:
Pattern value: 'PERMUTE(A, B, C)'
Pattern value: 'PERMUTE(A, B, C)'
DEBUG: _process_permute called with variables: ['A', 'B', 'C']
DEBUG: Token metadata: {'variables': ['A', 'B', 'C'], 'variable_count': 3, 'original': 'PERMUTE(A, B, C)', 'has_nested': False, 'has_alternations': False, 'nested': False, 'depth': 0}
DEBUG: Token type: PatternTokenType.PERMUTE


DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='A', target=6, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='B', target=8, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='C', target=10, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='A', target=13, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='B', target=15, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='C', target=17, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='A', target=20, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='C', target=22, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='B', target=24, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='B', target=27, pri

DEBUG: DFA metadata keys: ['has_permute', 'permute_variables', 'permute_structure', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: True
DEBUG: has_alternations: False
DEBUG: No alternation_combinations in DFA metadata
DEBUG: Parsed alternation_order: {}
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: False
DEBUG: has_complex_back_references: False
DEBUG: Processing row 0 with value N/A
DEBUG: Found 1 valid transitions: ['A']
DEBUG: Categorized transitions: {'accepting': [], 'prerequisite': [], 'simple': [('A', 2, False)], 'dependent': []}
DEBUG: Processing category '

DEBUG:row_match_recognize.src.matcher.matcher:    Condition failed for A
DEBUG:row_match_recognize.src.matcher.matcher:    DEBUG: condition result=False, type=<class 'bool'>
DEBUG:row_match_recognize.src.matcher.matcher:  [DEBUG] Clearing context.current_var (was A)
DEBUG:row_match_recognize.src.matcher.matcher:  Evaluating condition for var: B
DEBUG:row_match_recognize.src.matcher.matcher:  [DEBUG] Set context.current_var = B
DEBUG:row_match_recognize.src.matcher.matcher:    DEBUG: Calling condition function with row={'customer_id': 'cust_2', 'order_date': Timestamp('2020-05-18 00:00:00'), 'price': 6}
DEBUG:row_match_recognize.src.matcher.condition_evaluator:Universal pattern variable 'price' resolved to: 6
DEBUG:row_match_recognize.src.matcher.condition_evaluator:[DEBUG] COMPARE: left=6 (<class 'int'>), op=Eq
DEBUG:row_match_recognize.src.matcher.condition_evaluator:[DEBUG] COMPARE: left=6 (<class 'int'>), right=50 (<class 'int'>)
DEBUG:row_match_recognize.src.matcher.condition_evalu

  customer_id     a_date     b_date     c_date
0      cust_1 2020-05-14 2020-05-16 2020-05-12




In [9]:
import pandas as pd
from src.executor.match_recognize import match_recognize
import pandas as pd

# Define the data
data = [
    ('cust_1', '2020-05-11', 100),
    ('cust_1', '2020-05-12', 200),
    ('cust_2', '2020-05-13',   8),
    ('cust_1', '2020-05-14', 100),
    ('cust_2', '2020-05-15',   4),
    ('cust_1', '2020-05-16',  50),
    ('cust_1', '2020-05-17', 100),
    ('cust_2', '2020-05-18',   6),
]

# Create DataFrame
df = pd.DataFrame(data, columns=['customer_id', 'order_date', 'price'])

# Convert order_date column to datetime
df['order_date'] = pd.to_datetime(df['order_date'])

# Display the DataFrame
print(df)

query_basic = """
SELECT
  customer_id,
  start_price,
  last_b_price    AS peak_price,
  last_c_price    AS bottom_price,
  last_d_price    AS end_price,
  start_date,
  end_date
FROM memory.default.orders
MATCH_RECOGNIZE (
  PARTITION BY customer_id
  ORDER BY order_date
  MEASURES
    A.price               AS start_price,
    LAST(B.price)         AS last_b_price,
    LAST(C.price)         AS last_c_price,
    LAST(D.price)         AS last_d_price,
    A.order_date          AS start_date,
    LAST(D.order_date)    AS end_date
  PATTERN (A B+ C+ D+)
  DEFINE
    B AS B.price > PREV(price),
    C AS C.price < PREV(price),
    D AS D.price > PREV(price)
) AS t;

"""

print("Test 1:")
output_df = match_recognize(query_basic, df)
print(output_df)
print("\n")

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT customer_id, start_price, last_b_price AS peak_price, last_c_price AS bottom_price, last_d_price AS end_price, start_date, end_date FROM memory.default.orders MATCH_RECOGNIZE ( PARTITION BY customer_id ORDER BY order_date MEASURES A.price AS start_price, LAST(B.price) AS last_b_price, LAST(C.price) AS last_c_price, LAST(D.price) AS last_d_price, A.order_date AS start_date, LAST(D.order_date) AS end_date PATTERN (A B+ C+ D+) DEFINE B AS B.price > PREV(price), C AS C.price < PREV(price), D AS D.price > PREV(price) ) AS t;
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=customer_id, metadata={}), SelectItem(expression=start_price, metadata={}), SelectItem(expression=last_b_price, alias=peak_price, metadata={}), SelectItem(expression=last_c_price, alias=bottom_price, metadata={}), SelectItem(expression=last_d_price, alias=end_price, metadata={}), SelectItem(e

  customer_id order_date  price
0      cust_1 2020-05-11    100
1      cust_1 2020-05-12    200
2      cust_2 2020-05-13      8
3      cust_1 2020-05-14    100
4      cust_2 2020-05-15      4
5      cust_1 2020-05-16     50
6      cust_1 2020-05-17    100
7      cust_2 2020-05-18      6
Test 1:
Pattern value: 'A B+ C+ D+'
Pattern variables {'A'} have no DEFINE conditions - defaulting to TRUE (always match)
Pattern value: 'A B+ C+ D+'
Pattern variables {'A'} have no DEFINE conditions - defaulting to TRUE (always match)
DEBUG: DFA metadata keys: ['pattern_variables', 'variables_with_quantifiers', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: Fa

In [10]:

import pandas as pd
from src.executor.match_recognize import match_recognize
import pandas as pd

# Define the data
data = [
    ('cust_1', '2020-05-11', 100),
    ('cust_1', '2020-05-12', 200),
    ('cust_2', '2020-05-13',   8),
    ('cust_1', '2020-05-14', 100),
    ('cust_2', '2020-05-15',   4),
    ('cust_1', '2020-05-16',  50),
    ('cust_1', '2020-05-17', 100),
    ('cust_2', '2020-05-18',   6),
]

# Create DataFrame
df = pd.DataFrame(data, columns=['customer_id', 'order_date', 'price'])

# Convert order_date column to datetime
df['order_date'] = pd.to_datetime(df['order_date'])

# Display the DataFrame
print(df)

query_basic_permute = """
SELECT customer_id, start_price, bottom_price, final_price, start_date, final_date
    FROM memory.default.orders
        MATCH_RECOGNIZE (
            PARTITION BY customer_id
            ORDER BY order_date
            MEASURES
                START.price AS start_price,
                LAST(DOWN.price) AS bottom_price,
                LAST(UP.price) AS final_price,
                START.order_date AS start_date,
                LAST(UP.order_date) AS final_date
            ONE ROW PER MATCH
            AFTER MATCH SKIP PAST LAST ROW
            PATTERN (START DOWN+ UP+)
            DEFINE
                DOWN AS price < PREV(price),
                UP AS price > PREV(price)
            );
"""

print("Test 1: Basic PERMUTE - Should match all sequences with A, B, C in any order")
output_df = match_recognize(query_basic_permute, df)
print(output_df)
print("\n")

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT customer_id, start_price, bottom_price, final_price, start_date, final_date FROM memory.default.orders MATCH_RECOGNIZE ( PARTITION BY customer_id ORDER BY order_date MEASURES START.price AS start_price, LAST(DOWN.price) AS bottom_price, LAST(UP.price) AS final_price, START.order_date AS start_date, LAST(UP.order_date) AS final_date ONE ROW PER MATCH AFTER MATCH SKIP PAST LAST ROW PATTERN (START DOWN+ UP+) DEFINE DOWN AS price < PREV(price), UP AS price > PREV(price) );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=customer_id, metadata={}), SelectItem(expression=start_price, metadata={}), SelectItem(expression=bottom_price, metadata={}), SelectItem(expression=final_price, metadata={}), SelectItem(expression=start_date, metadata={}), SelectItem(expression=final_date, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromCla

  customer_id order_date  price
0      cust_1 2020-05-11    100
1      cust_1 2020-05-12    200
2      cust_2 2020-05-13      8
3      cust_1 2020-05-14    100
4      cust_2 2020-05-15      4
5      cust_1 2020-05-16     50
6      cust_1 2020-05-17    100
7      cust_2 2020-05-18      6
Test 1: Basic PERMUTE - Should match all sequences with A, B, C in any order
Pattern value: 'START DOWN+ UP+'
Pattern variables {'START'} have no DEFINE conditions - defaulting to TRUE (always match)
Pattern value: 'START DOWN+ UP+'
Pattern variables {'START'} have no DEFINE conditions - defaulting to TRUE (always match)
DEBUG: DFA metadata keys: ['pattern_variables', 'variables_with_quantifiers', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transiti

# Example #2

In [11]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Create test data with different permutation patterns
data = [
    # Sequence 1: Has A-B-C pattern
    {"id": 1, "seq": 1, "step": 1, "event_type": "start", "value": 100},  # A
    {"id": 2, "seq": 1, "step": 2, "event_type": "middle", "value": 200}, # B
    {"id": 3, "seq": 1, "step": 3, "event_type": "end", "value": 300},    # C
    
    # Sequence 2: Has B-A-C pattern
    {"id": 4, "seq": 2, "step": 1, "event_type": "middle", "value": 250}, # B
    {"id": 5, "seq": 2, "step": 2, "event_type": "start", "value": 150},  # A
    {"id": 6, "seq": 2, "step": 3, "event_type": "end", "value": 350},    # C
    
    # Sequence 3: Has A-C-B pattern
    {"id": 7, "seq": 3, "step": 1, "event_type": "start", "value": 175},  # A
    {"id": 8, "seq": 3, "step": 2, "event_type": "end", "value": 275},    # C
    {"id": 9, "seq": 3, "step": 3, "event_type": "middle", "value": 375}, # B
    
    # Sequence 4: Has C-B-A pattern
    {"id": 10, "seq": 4, "step": 1, "event_type": "end", "value": 225},   # C
    {"id": 11, "seq": 4, "step": 2, "event_type": "middle", "value": 325}, # B
    {"id": 12, "seq": 4, "step": 3, "event_type": "start", "value": 425},  # A
]

df = pd.DataFrame(data)

print("Testing PERMUTE Patterns\n")

# Test 1: Basic PERMUTE - Match any order of A, B, C
query_basic_permute = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        A.value AS a_value,
        B.value AS b_value,
        C.value AS c_value
    ONE ROW PER MATCH
    PATTERN (PERMUTE(A, B, C))
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

print("Test 1: Basic PERMUTE - Should match all sequences with A, B, C in any order")
output_df = match_recognize(query_basic_permute, df)
print(output_df)
print("\n")

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.op2 MATCH_RECOGNIZE( PARTITION BY seq ORDER BY step MEASURES CLASSIFIER() AS pattern_var, MATCH_NUMBER() AS match_num, A.value AS a_value, B.value AS b_value, C.value AS c_value ONE ROW PER MATCH PATTERN (PERMUTE(A, B, C)) DEFINE A AS event_type = 'start', B AS event_type = 'middle', C AS event_type = 'end' );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['seq'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='step', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extracto

Testing PERMUTE Patterns

Test 1: Basic PERMUTE - Should match all sequences with A, B, C in any order
Pattern value: 'PERMUTE(A, B, C)'
Pattern value: 'PERMUTE(A, B, C)'
DEBUG: _process_permute called with variables: ['A', 'B', 'C']
DEBUG: Token metadata: {'variables': ['A', 'B', 'C'], 'variable_count': 3, 'original': 'PERMUTE(A, B, C)', 'has_nested': False, 'has_alternations': False, 'nested': False, 'depth': 0}
DEBUG: Token type: PatternTokenType.PERMUTE


DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='A', target=6, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='B', target=8, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='C', target=10, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='A', target=13, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='B', target=15, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='C', target=17, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='A', target=20, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='C', target=22, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='B', target=24, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='B', target=27, pri

DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='A', target=38, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='C', target=41, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='A', target=43, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='B', target=45, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='C', target=48, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='B', target=50, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='A', target=52, priority=0
DEBUG:row_match_recognize.performance:Starting epsilon_closure
INFO:row_match_recognize.performance:epsilon_closure completed in 0.0003s
DEBUG:row_match_recognize.src.matcher.automata:Epsilon closure of [0] = [0, 2, 3, 5, 6, 12, 13, 19, 20, 26, 27, 33, 34, 40, 41] (iterations: 15)
DEBUG:row_m

DEBUG: DFA metadata keys: ['has_permute', 'permute_variables', 'permute_structure', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: True
DEBUG: has_alternations: False
DEBUG: No alternation_combinations in DFA metadata
DEBUG: Parsed alternation_order: {}
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: False
DEBUG: has_complex_back_references: False
DEBUG: Processing row 0 with value 100
DEBUG: Found 1 valid transitions: ['A']
DEBUG: Categorized transitions: {'accepting': [], 'prerequisite': [], 'simple': [('A', 2, False)], 'dependent': []}
DEBUG: Processing category '

DEBUG:row_match_recognize.src.matcher.measure_evaluator:Context variables: {'A': [0], 'C': [1], 'B': [2]}
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Number of rows: 3
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Current index: 2
INFO:row_match_recognize.src.matcher.production_aggregates:ENHANCED_EVAL_DEBUG: Original evaluator returned: B (type: <class 'str'>)
DEBUG:row_match_recognize.src.matcher.matcher:Setting pattern_var to B from evaluator
DEBUG:row_match_recognize.src.matcher.production_aggregates:Enhanced evaluate called with expr: 'MATCH_NUMBER()', semantics: 'FINAL'
DEBUG:row_match_recognize.src.matcher.production_aggregates:Pattern match result for 'MATCH_NUMBER()': <re.Match object; span=(0, 13), match='MATCH_NUMBER('>
DEBUG:row_match_recognize.src.matcher.production_aggregates:Matched function: MATCH_NUMBER, semantics_prefix: None
DEBUG:row_match_recognize.src.matcher.production_aggregates:Function MATCH_NUMBER not in standard aggregates, using origin

DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: False
DEBUG: has_complex_back_references: False
DEBUG: Processing row 0 with value 225
DEBUG: Found 1 valid transitions: ['C']
DEBUG: Categorized transitions: {'accepting': [], 'prerequisite': [], 'simple': [('C', 1, False)], 'dependent': []}
DEBUG: Processing category 'simple' with 1 transitions
DEBUG: Selected simple transition: C -> state 1 (alternation priority: N/A)
DEBUG: Processing row 1 with value 325
DEBUG: Found 1 valid transitions: ['B']
DEBUG: Categorized transitions: {'accepting': [], 'prerequisite': [], 'simple': [('B', 4, False)], 'dependent': []}
DEBUG: Processing category 'simple' with 1 transitions
DEBUG: Selected simple transition: B -> state 4 (alternation priority: N/A)
DEBUG: Processing row 2 with value 425
DEBUG: Found 1 valid transitions: ['A']
DEBUG: Categorized transitions: {'accepting': [('A', 10, False)], 'prerequisite': [], 'simple':

DEBUG:row_match_recognize.src.executor.match_recognize:Setting a_value to 100 from evaluator
DEBUG:row_match_recognize.src.matcher.production_aggregates:Enhanced evaluate called with expr: 'B.value', semantics: 'FINAL'
DEBUG:row_match_recognize.src.matcher.production_aggregates:Pattern match result for 'B.value': None
DEBUG:row_match_recognize.src.matcher.production_aggregates:Expression doesn't match pure aggregate pattern: B.value
INFO:row_match_recognize.src.matcher.production_aggregates:ENHANCED_EVAL_DEBUG: Falling back to original evaluator for: B.value
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Evaluating expression: B.value with FINAL semantics
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Context variables: {'A': [0], 'B': [1], 'C': [2]}
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Number of rows: 12
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Current index: 2
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Evaluating patter

   seq  step pattern_var  match_num  a_value  b_value  c_value
0    1     1           C          1      100      200      300
1    2     1           C          1      150      250      350
2    3     1           B          1      175      375      275
3    4     1           A          1      425      325      225




In [12]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Create test data with different permutation patterns
data = [
    # Sequence 1: Has A-B-C pattern
    {"id": 1, "seq": 1, "step": 1, "event_type": "start", "value": 100},  # A
    {"id": 2, "seq": 1, "step": 2, "event_type": "middle", "value": 200}, # B
    {"id": 3, "seq": 1, "step": 3, "event_type": "end", "value": 300},    # C
    
    # Sequence 2: Has B-A-C pattern
    {"id": 4, "seq": 2, "step": 1, "event_type": "middle", "value": 250}, # B
    {"id": 5, "seq": 2, "step": 2, "event_type": "start", "value": 150},  # A
    {"id": 6, "seq": 2, "step": 3, "event_type": "end", "value": 350},    # C
    
    # Sequence 3: Has A-C-B pattern
    {"id": 7, "seq": 3, "step": 1, "event_type": "start", "value": 175},  # A
    {"id": 8, "seq": 3, "step": 2, "event_type": "end", "value": 275},    # C
    {"id": 9, "seq": 3, "step": 3, "event_type": "middle", "value": 375}, # B
    
    # Sequence 4: Has C-B-A pattern
    {"id": 10, "seq": 4, "step": 1, "event_type": "end", "value": 225},   # C
    {"id": 11, "seq": 4, "step": 2, "event_type": "middle", "value": 325}, # B
    {"id": 12, "seq": 4, "step": 3, "event_type": "start", "value": 425},  # A
]

df = pd.DataFrame(data)

print("Testing PERMUTE Patterns\n")

# Test 1: Basic PERMUTE - Match any order of A, B, C
query_basic_permute = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        A.value AS a_value,
        B.value AS b_value,
        C.value AS c_value
  
    PATTERN (PERMUTE(A, B, C))
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

print("Test 1: Basic PERMUTE - Should match all sequences with A, B, C in any order")
output_df = match_recognize(query_basic_permute, df)
print(output_df)
print("\n")

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.op2 MATCH_RECOGNIZE( PARTITION BY seq ORDER BY step MEASURES CLASSIFIER() AS pattern_var, MATCH_NUMBER() AS match_num, A.value AS a_value, B.value AS b_value, C.value AS c_value PATTERN (PERMUTE(A, B, C)) DEFINE A AS event_type = 'start', B AS event_type = 'middle', C AS event_type = 'end' );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['seq'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='step', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASUR

Testing PERMUTE Patterns

Test 1: Basic PERMUTE - Should match all sequences with A, B, C in any order
Pattern value: 'PERMUTE(A, B, C)'
Pattern value: 'PERMUTE(A, B, C)'
DEBUG: DFA metadata keys: ['has_permute', 'permute_variables', 'permute_structure', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: True
DEBUG: has_alternations: False
DEBUG: No alternation_combinations in DFA metadata
DEBUG: Parsed alternation_order: {}
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: False
DEBUG: has_complex_back_references: False
DEBUG: Processing row 0 with value 100
DEBUG: Found 

DEBUG:row_match_recognize.src.matcher.measure_evaluator:Evaluating expression: C.value with FINAL semantics
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Context variables: {'A': [0], 'B': [1], 'C': [2]}
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Number of rows: 12
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Current index: 2
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Evaluating pattern variable reference: C.value, current_idx=2, is_running=False, is_permute=True
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Pattern variable reference matched: var=C, col=value
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Using PERMUTE-specific logic for C
DEBUG:row_match_recognize.src.matcher.measure_evaluator:PERMUTE match found value 300 at index 2
INFO:row_match_recognize.src.matcher.production_aggregates:ENHANCED_EVAL_DEBUG: Original evaluator returned: 300 (type: <class 'int'>)
DEBUG:row_match_recognize.src.executor.match_recogniz

   seq  step pattern_var  match_num  a_value  b_value  c_value
0    1     1           C          1      100      200      300
1    2     1           C          1      150      250      350
2    3     1           B          1      175      375      275
3    4     1           A          1      425      325      225




In [13]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Create test data with different permutation patterns
data = [
    # Sequence 1: Has A-B-C pattern
    {"id": 1, "seq": 1, "step": 1, "event_type": "start", "value": 100},  # A
    {"id": 2, "seq": 1, "step": 2, "event_type": "middle", "value": 200}, # B
    {"id": 3, "seq": 1, "step": 3, "event_type": "end", "value": 300},    # C
    
    # Sequence 2: Has B-A-C pattern
    {"id": 4, "seq": 2, "step": 1, "event_type": "middle", "value": 250}, # B
    {"id": 5, "seq": 2, "step": 2, "event_type": "start", "value": 150},  # A
    {"id": 6, "seq": 2, "step": 3, "event_type": "end", "value": 350},    # C
    
    # Sequence 3: Has A-C-B pattern
    {"id": 7, "seq": 3, "step": 1, "event_type": "start", "value": 175},  # A
    {"id": 8, "seq": 3, "step": 2, "event_type": "end", "value": 275},    # C
    {"id": 9, "seq": 3, "step": 3, "event_type": "middle", "value": 375}, # B
    
    # Sequence 4: Has C-B-A pattern
    {"id": 10, "seq": 4, "step": 1, "event_type": "end", "value": 225},   # C
    {"id": 11, "seq": 4, "step": 2, "event_type": "middle", "value": 325}, # B
    {"id": 12, "seq": 4, "step": 3, "event_type": "start", "value": 425},  # A
]

df = pd.DataFrame(data)

print("Testing PERMUTE Patterns\n")
# Test 2: PERMUTE with Quantifier
query_permute_quantifier = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        FIRST(A.value) AS first_a_value,
        LAST(C.value) AS last_c_value
    ONE ROW PER MATCH
    PATTERN (PERMUTE(A, B, C)+)
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

print("Test 2: PERMUTE with Quantifier - Should match one or more occurrences of permutations")
output_df = match_recognize(query_permute_quantifier, df)
print(output_df)
print("\n")

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.op2 MATCH_RECOGNIZE( PARTITION BY seq ORDER BY step MEASURES CLASSIFIER() AS pattern_var, MATCH_NUMBER() AS match_num, FIRST(A.value) AS first_a_value, LAST(C.value) AS last_c_value ONE ROW PER MATCH PATTERN (PERMUTE(A, B, C)+) DEFINE A AS event_type = 'start', B AS event_type = 'middle', C AS event_type = 'end' );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['seq'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='step', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_ext

Testing PERMUTE Patterns

Test 2: PERMUTE with Quantifier - Should match one or more occurrences of permutations
Pattern value: 'PERMUTE(A, B, C)+'
Pattern value: 'PERMUTE(A, B, C)+'
DEBUG: _process_permute called with variables: ['A', 'B', 'C']
DEBUG: Token metadata: {'variables': ['A', 'B', 'C'], 'variable_count': 3, 'original': 'PERMUTE(A, B, C)', 'has_nested': False, 'has_alternations': False, 'nested': False, 'depth': 0}
DEBUG: Token type: PatternTokenType.PERMUTE


DEBUG:row_match_recognize.src.matcher.automata:Processing quantified PERMUTE: PERMUTE(A, B, C) with quantifier +


DEBUG: _process_permute called with variables: ['A', 'B', 'C']
DEBUG: Token metadata: {'variables': ['A', 'B', 'C'], 'variable_count': 3, 'original': 'PERMUTE(A, B, C)', 'has_nested': False, 'has_alternations': False, 'nested': False, 'depth': 0}
DEBUG: Token type: PatternTokenType.PERMUTE


DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='A', target=8, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='B', target=10, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='C', target=12, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='A', target=15, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='B', target=17, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='C', target=19, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='A', target=22, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='C', target=24, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='B', target=26, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='B', target=29, pr

...

DEBUG: Found 1 valid transitions: ['C']
DEBUG: Categorized transitions: {'accepting': [('C', 15, False)], 'prerequisite': [], 'simple': [], 'dependent': []}
DEBUG: Processing category 'accepting' with 1 transitions
DEBUG: Selected accepting transition: C -> state 15 (alternation priority: N/A)
DEBUG: _find_single_match called with start_idx=0


DEBUG:row_match_recognize.src.matcher.matcher:Checking complex back-references for 3 conditions
DEBUG:row_match_recognize.src.matcher.matcher:Analyzing condition for A: event_type = 'start'
DEBUG:row_match_recognize.src.matcher.matcher:  Referenced vars: set()
DEBUG:row_match_recognize.src.matcher.matcher:Analyzing condition for B: event_type = 'middle'
DEBUG:row_match_recognize.src.matcher.matcher:  Referenced vars: set()
DEBUG:row_match_recognize.src.matcher.matcher:Analyzing condition for C: event_type = 'end'
DEBUG:row_match_recognize.src.matcher.matcher:  Referenced vars: set()
DEBUG:row_match_recognize.src.matcher.matcher:Summary: has_nav_functions=False, cross_var_dependencies=False, has_classifier_subset_refs=False
DEBUG:row_match_recognize.src.matcher.matcher:No complex back-references detected
DEBUG:row_match_recognize.src.matcher.matcher:Checking complex back-references for 3 conditions
DEBUG:row_match_recognize.src.matcher.matcher:Analyzing condition for A: event_type = 'st

DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: False
DEBUG: has_complex_back_references: False
DEBUG: Processing row 0 with value 175
DEBUG: Found 1 valid transitions: ['A']
DEBUG: Categorized transitions: {'accepting': [], 'prerequisite': [], 'simple': [('A', 2, False)], 'dependent': []}
DEBUG: Processing category 'simple' with 1 transitions
DEBUG: Selected simple transition: A -> state 2 (alternation priority: N/A)
DEBUG: Processing row 1 with value 275
DEBUG: Found 1 valid transitions: ['C']
DEBUG: Categorized transitions: {'accepting': [], 'prerequisite': [], 'simple': [('C', 6, False)], 'dependent': []}
DEBUG: Processing category 'simple' with 1 transitions
DEBUG: Selected simple transition: C -> state 6 (alternation priority: N/A)
DEBUG: Processing row 2 with value 375
DEBUG: Found 1 valid transitions: ['B']
DEBUG: Categorized transitions: {'accepting': [('B', 12, False)], 'prerequisite': [], 'simple': [], 'dependent': []}
DEBUG: Processing category '

DEBUG:row_match_recognize.src.matcher.measure_evaluator:Number of rows: 12
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Current index: 8
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Evaluating pattern variable reference: FIRST(A.value), current_idx=8, is_running=False, is_permute=True
INFO:row_match_recognize.src.matcher.production_aggregates:ENHANCED_EVAL_DEBUG: Original evaluator returned: 175 (type: <class 'int'>)
DEBUG:row_match_recognize.src.executor.match_recognize:Setting first_a_value to 175 from evaluator
DEBUG:row_match_recognize.src.matcher.production_aggregates:Enhanced evaluate called with expr: 'LAST(C.value)', semantics: 'FINAL'
DEBUG:row_match_recognize.src.matcher.production_aggregates:Pattern match result for 'LAST(C.value)': <re.Match object; span=(0, 5), match='LAST('>
DEBUG:row_match_recognize.src.matcher.production_aggregates:Matched function: LAST, semantics_prefix: None
DEBUG:row_match_recognize.src.matcher.production_aggregates:Function LA

   seq  step pattern_var  match_num  first_a_value  last_c_value
0    1     1           C          1            100           300
1    2     1           C          1            150           350
2    3     1           B          1            175           275
3    4     1           A          1            425           225




In [16]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Create test data with different permutation patterns
data = [
    # Sequence 1: Has A-B-C pattern
    {"id": 1, "seq": 1, "step": 1, "event_type": "start", "value": 100},  # A
    {"id": 2, "seq": 1, "step": 2, "event_type": "middle", "value": 200}, # B
    {"id": 3, "seq": 1, "step": 3, "event_type": "end", "value": 300},    # C
    
    # Sequence 2: Has B-A-C pattern
    {"id": 4, "seq": 2, "step": 1, "event_type": "middle", "value": 250}, # B
    {"id": 5, "seq": 2, "step": 2, "event_type": "start", "value": 150},  # A
    {"id": 6, "seq": 2, "step": 3, "event_type": "end", "value": 350},    # C
    
    # Sequence 3: Has A-C-B pattern
    {"id": 7, "seq": 3, "step": 1, "event_type": "start", "value": 175},  # A
    {"id": 8, "seq": 3, "step": 2, "event_type": "end", "value": 275},    # C
    {"id": 9, "seq": 3, "step": 3, "event_type": "middle", "value": 375}, # B
    
    # Sequence 4: Has C-B-A pattern
    {"id": 10, "seq": 4, "step": 1, "event_type": "end", "value": 225},   # C
    {"id": 11, "seq": 4, "step": 2, "event_type": "middle", "value": 325}, # B
    {"id": 12, "seq": 4, "step": 3, "event_type": "start", "value": 425},  # A
]

df = pd.DataFrame(data)

print("Testing PERMUTE Patterns\n")
# Test 3: PERMUTE with ALL ROWS PER MATCH
query_permute_all_rows = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        RUNNING LAST(A.value) AS running_a_value
    ALL ROWS PER MATCH
    PATTERN (PERMUTE(A, B, C))
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

print("Test 3: PERMUTE with ALL ROWS PER MATCH - Shows all matched rows")
output_df = match_recognize(query_permute_all_rows, df)
print(output_df)
print("\n")


Testing PERMUTE Patterns

Test 3: PERMUTE with ALL ROWS PER MATCH - Shows all matched rows


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.op2 MATCH_RECOGNIZE( PARTITION BY seq ORDER BY step MEASURES CLASSIFIER() AS pattern_var, MATCH_NUMBER() AS match_num, RUNNING LAST(A.value) AS running_a_value ALL ROWS PER MATCH PATTERN (PERMUTE(A, B, C)) DEFINE A AS event_type = 'start', B AS event_type = 'middle', C AS event_type = 'end' );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['seq'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='step', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASU

Pattern value: 'PERMUTE(A, B, C)'
Pattern value: 'PERMUTE(A, B, C)'
Creating transition for variable 'A' with condition: 'event_type = 'start''
Creating transition for variable 'B' with condition: 'event_type = 'middle''
Creating transition for variable 'C' with condition: 'event_type = 'end''
Initialized matcher with excluded variables: set()
Find matches with all_rows=True, show_empty=True, include_unmatched=False
Starting match at index 0, state: State 0 (Non-accept, Vars: A, B, C)
Testing row 0, data: {'id': 1, 'seq': 1, 'step': 1, 'event_type': 'start', 'value': 100}
  Evaluating condition for var: A
    Condition passed for A
  Assigned row 0 to variable A
Reached accepting state 1 at row 0
  Current longest match: 0-0, vars: ['A']
Testing row 1, data: {'id': 2, 'seq': 1, 'step': 2, 'event_type': 'middle', 'value': 200}
  Evaluating condition for var: B
    Condition passed for B
  Assigned row 1 to variable B
Reached accepting state 2 at row 1
  Current longest match: 0-1, vars:

# check_fix


In [15]:

import pandas as pd
from src.executor.match_recognize import match_recognize

# Create test data with different permutation patterns
data = [
    # Sequence 1: Has A-B-C pattern
    {"id": 1, "seq": 1, "step": 1, "event_type": "start", "value": 100},  # A
    {"id": 2, "seq": 1, "step": 2, "event_type": "middle", "value": 200}, # B
    {"id": 3, "seq": 1, "step": 3, "event_type": "end", "value": 300},    # C
    
    # Sequence 2: Has B-A-C pattern
    {"id": 4, "seq": 2, "step": 1, "event_type": "middle", "value": 250}, # B
    {"id": 5, "seq": 2, "step": 2, "event_type": "start", "value": 150},  # A
    {"id": 6, "seq": 2, "step": 3, "event_type": "end", "value": 350},    # C
    
    # Sequence 3: Has A-C-B pattern
    {"id": 7, "seq": 3, "step": 1, "event_type": "start", "value": 175},  # A
    {"id": 8, "seq": 3, "step": 2, "event_type": "end", "value": 275},    # C
    {"id": 9, "seq": 3, "step": 3, "event_type": "middle", "value": 375}, # B
    
    # Sequence 4: Has C-B-A pattern
    {"id": 10, "seq": 4, "step": 1, "event_type": "end", "value": 225},   # C
    {"id": 11, "seq": 4, "step": 2, "event_type": "middle", "value": 325}, # B
    {"id": 12, "seq": 4, "step": 3, "event_type": "start", "value": 425},  # A
]

df = pd.DataFrame(data)

print("Testing PERMUTE Patterns\n")
# Test 4: PERMUTE with Subset Variables
query_permute_subset = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        X.value AS x_value,
        Y.value AS y_value
    ONE ROW PER MATCH
    PATTERN (PERMUTE(A, B, C))
    SUBSET
        X = (A, B),
        Y = (B, C)
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

print("Test 4: PERMUTE with Subset Variables - Using subset groupings")
output_df = match_recognize(query_permute_subset, df)
print(output_df)
print("\n")


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.op2 MATCH_RECOGNIZE( PARTITION BY seq ORDER BY step MEASURES CLASSIFIER() AS pattern_var, MATCH_NUMBER() AS match_num, X.value AS x_value, Y.value AS y_value ONE ROW PER MATCH PATTERN (PERMUTE(A, B, C)) SUBSET X = (A, B), Y = (B, C) DEFINE A AS event_type = 'start', B AS event_type = 'middle', C AS event_type = 'end' );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['seq'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='step', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recogniz

Testing PERMUTE Patterns

Test 4: PERMUTE with Subset Variables - Using subset groupings
Pattern value: 'PERMUTE(A, B, C)'
Extracted subset definition: X = (A, B)
Extracted subset definition: Y = (B, C)
Pattern value: 'PERMUTE(A, B, C)'
DEBUG: DFA metadata keys: ['has_permute', 'permute_variables', 'permute_structure', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: True
DEBUG: has_alternations: False
DEBUG: No alternation_combinations in DFA metadata
DEBUG: Parsed alternation_order: {}
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: False
DEBUG: has_complex_back_refe

DEBUG:row_match_recognize.src.executor.match_recognize:Measures: ['pattern_var', 'match_num', 'x_value', 'y_value']
DEBUG:row_match_recognize.src.executor.match_recognize:Rows per match mode: RowsPerMatch.ONE_ROW
INFO:row_match_recognize.src.executor.match_recognize:Query execution metrics: {'parsing_time': 0.009966611862182617, 'automata_build_time': 0.0013935565948486328, 'matching_time': 0.15102434158325195, 'result_processing_time': 0.04270362854003906, 'total_time': 0.20658516883850098, 'partition_count': 4, 'match_count': 4}
INFO:row_match_recognize.src.executor.match_recognize:Pattern cache statistics: {'total_hits': 5, 'total_misses': 10, 'total_compilation_time_saved': 0.22166013717651367, 'memory_used_mb': 5.0}
INFO:row_match_recognize.src.executor.match_recognize:Cache hit rate: 33.3% (5/15)
INFO:row_match_recognize.src.executor.match_recognize:Compilation time saved: 0.222s
INFO:row_match_recognize.src.executor.match_recognize:Cache memory usage: 5.00 MB


   seq  step pattern_var  match_num  x_value  y_value
0    1     1           C          1      100      200
1    2     1           C          1      150      250
2    3     1           B          1      175      375
3    4     1           A          1      425      325




In [17]:

import pandas as pd
from src.executor.match_recognize import match_recognize

# Create test data with different permutation patterns
data = [
    # Sequence 1: Has A-B-C pattern
    {"id": 1, "seq": 1, "step": 1, "event_type": "start", "value": 100},  # A
    {"id": 2, "seq": 1, "step": 2, "event_type": "middle", "value": 200}, # B
    {"id": 3, "seq": 1, "step": 3, "event_type": "end", "value": 300},    # C
    
    # Sequence 2: Has B-A-C pattern
    {"id": 4, "seq": 2, "step": 1, "event_type": "middle", "value": 250}, # B
    {"id": 5, "seq": 2, "step": 2, "event_type": "start", "value": 150},  # A
    {"id": 6, "seq": 2, "step": 3, "event_type": "end", "value": 350},    # C
    
    # Sequence 3: Has A-C-B pattern
    {"id": 7, "seq": 3, "step": 1, "event_type": "start", "value": 175},  # A
    {"id": 8, "seq": 3, "step": 2, "event_type": "end", "value": 275},    # C
    {"id": 9, "seq": 3, "step": 3, "event_type": "middle", "value": 375}, # B
    
    # Sequence 4: Has C-B-A pattern
    {"id": 10, "seq": 4, "step": 1, "event_type": "end", "value": 225},   # C
    {"id": 11, "seq": 4, "step": 2, "event_type": "middle", "value": 325}, # B
    {"id": 12, "seq": 4, "step": 3, "event_type": "start", "value": 425},  # A
]

df = pd.DataFrame(data)

print("Testing PERMUTE Patterns\n")
# Test 5: Nested PERMUTE patterns
query_nested_permute = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        A.value AS a_value,
        B.value AS b_value,
        C.value AS c_value
    ONE ROW PER MATCH
    PATTERN (PERMUTE(A, PERMUTE(B, C)))
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

print("Test 5: Nested PERMUTE - Testing nested permutation patterns")
output_df = match_recognize(query_nested_permute, df)
print(output_df)
print("\n")

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.op2 MATCH_RECOGNIZE( PARTITION BY seq ORDER BY step MEASURES CLASSIFIER() AS pattern_var, MATCH_NUMBER() AS match_num, A.value AS a_value, B.value AS b_value, C.value AS c_value ONE ROW PER MATCH PATTERN (PERMUTE(A, PERMUTE(B, C))) DEFINE A AS event_type = 'start', B AS event_type = 'middle', C AS event_type = 'end' );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['seq'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='step', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize

Testing PERMUTE Patterns

Test 5: Nested PERMUTE - Testing nested permutation patterns
Pattern value: 'PERMUTE(A, PERMUTE(B, C))'
Pattern value: 'PERMUTE(A, PERMUTE(B, C))'
DEBUG: DFA metadata keys: ['has_permute', 'permute_variables', 'permute_structure', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: True
DEBUG: has_alternations: False
DEBUG: No alternation_combinations in DFA metadata
DEBUG: Parsed alternation_order: {}
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: False
DEBUG: has_complex_back_references: False
DEBUG: Processing row 0 with value 100
DEBUG: Foun

INFO:row_match_recognize.src.matcher.matcher:Match start: 2, end: 2
INFO:row_match_recognize.src.matcher.matcher:Variables: {'A': [2]}
INFO:row_match_recognize.src.matcher.matcher:
Result row:
INFO:row_match_recognize.src.matcher.matcher:pattern_var: A
INFO:row_match_recognize.src.matcher.matcher:match_num: 1
INFO:row_match_recognize.src.matcher.matcher:a_value: 425
INFO:row_match_recognize.src.matcher.matcher:b_value: None
INFO:row_match_recognize.src.matcher.matcher:c_value: None
INFO:row_match_recognize.src.matcher.matcher:MATCH_NUMBER: 1
INFO:row_match_recognize.src.matcher.matcher:_original_row_idx: 2
DEBUG:row_match_recognize.src.matcher.matcher:Calculating skip position: mode=SkipMode.PAST_LAST_ROW, skip_var=None, match_range=[2:2]
DEBUG:row_match_recognize.src.matcher.matcher:PAST_LAST_ROW: skipping to position 3
DEBUG:row_match_recognize.src.matcher.matcher:Non-empty match, advancing from 2 to 3
DEBUG:row_match_recognize.src.matcher.matcher:SKIP PAST LAST ROW: continuing searc

Empty DataFrame
Columns: [seq, pattern_var, match_num, a_value, b_value, c_value]
Index: []




In [18]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Create test data with different permutation patterns
data = [
    # Sequence 1: Has A-B-C pattern
    {"id": 1, "seq": 1, "step": 1, "event_type": "start", "value": 100},  # A
    {"id": 2, "seq": 1, "step": 2, "event_type": "middle", "value": 200}, # B
    {"id": 3, "seq": 1, "step": 3, "event_type": "end", "value": 300},    # C
    
    # Sequence 2: Has B-A-C pattern
    {"id": 4, "seq": 2, "step": 1, "event_type": "middle", "value": 250}, # B
    {"id": 5, "seq": 2, "step": 2, "event_type": "start", "value": 150},  # A
    {"id": 6, "seq": 2, "step": 3, "event_type": "end", "value": 350},    # C
    
    # Sequence 3: Has A-C-B pattern
    {"id": 7, "seq": 3, "step": 1, "event_type": "start", "value": 175},  # A
    {"id": 8, "seq": 3, "step": 2, "event_type": "end", "value": 275},    # C
    {"id": 9, "seq": 3, "step": 3, "event_type": "middle", "value": 375}, # B
    
    # Sequence 4: Has C-B-A pattern
    {"id": 10, "seq": 4, "step": 1, "event_type": "end", "value": 225},   # C
    {"id": 11, "seq": 4, "step": 2, "event_type": "middle", "value": 325}, # B
    {"id": 12, "seq": 4, "step": 3, "event_type": "start", "value": 425},  # A
]

df = pd.DataFrame(data)

print("Testing PERMUTE Patterns\n")
# Test 6: PERMUTE with Complex Conditions
query_permute_complex = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        A.value AS start_value,
        B.value AS middle_value,
        C.value AS end_value
    ONE ROW PER MATCH
    PATTERN (PERMUTE(A, B, C))
    DEFINE 
        A AS event_type = 'start' AND A.value < NEXT(A.value),
        B AS event_type = 'middle' AND B.value > PREV(B.value),
        C AS event_type = 'end' AND C.value > FIRST(A.value)
);
"""

print("Test 6: PERMUTE with Complex Conditions - Testing complex pattern definitions")
output_df = match_recognize(query_permute_complex, df)
print(output_df)
print("\n")

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.op2 MATCH_RECOGNIZE( PARTITION BY seq ORDER BY step MEASURES CLASSIFIER() AS pattern_var, MATCH_NUMBER() AS match_num, A.value AS start_value, B.value AS middle_value, C.value AS end_value ONE ROW PER MATCH PATTERN (PERMUTE(A, B, C)) DEFINE A AS event_type = 'start' AND A.value < NEXT(A.value), B AS event_type = 'middle' AND B.value > PREV(B.value), C AS event_type = 'end' AND C.value > FIRST(A.value) );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')


DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['seq'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='step', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='CLASSIFIER()', alias='pattern_var', metadata={'semantics': 'FINAL', 'explicit_semantics': False}, is_classifier=True, is_match_number=False), Measure(expression='MATCH_NUMBER()', alias='match_num', metadata={'semantics': 'FINAL', 'explicit_semantics': False}, is_classifier=False, is_match_number=True), Measure(expression='A.value', alias='start_value', metadata={'semantics': 'FINAL', 'explicit_semantics': False}, is_classifier=False, is_match_number=False), Measure(expression='B.value', alias='middle_value', metadata={'semantics': 'FINAL', 'explicit_sem

Testing PERMUTE Patterns

Test 6: PERMUTE with Complex Conditions - Testing complex pattern definitions
Pattern value: 'PERMUTE(A, B, C)'
Pattern value: 'PERMUTE(A, B, C)'
DEBUG: _process_permute called with variables: ['A', 'B', 'C']
DEBUG: Token metadata: {'variables': ['A', 'B', 'C'], 'variable_count': 3, 'original': 'PERMUTE(A, B, C)', 'has_nested': False, 'has_alternations': False, 'nested': False, 'depth': 0}
DEBUG: Token type: PatternTokenType.PERMUTE


DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='A', target=6, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='B', target=8, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='C', target=10, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='A', target=13, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='B', target=15, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='C', target=17, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='A', target=20, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='C', target=22, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='B', target=24, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='B', target=27, pri

DEBUG: DFA metadata keys: ['has_permute', 'permute_variables', 'permute_structure', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: True
DEBUG: has_alternations: False
DEBUG: No alternation_combinations in DFA metadata
DEBUG: Parsed alternation_order: {}
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: True
DEBUG: Using backtracking matcher for complex pattern
DEBUG: Starting backtracking search with 3 rows, max_iterations=5000
DEBUG: Before sorting, 3 successors found
DEBUG: Successor 0: vars=['C'], last_var=C, row=1
DEBUG: Successor 1: vars=['A'], last_var=A, row=1
D

DEBUG:row_match_recognize.src.matcher.condition_evaluator:[DEBUG] COMPARE RESULT: 175 Lt 275 = True (evaluating for var=A)
DEBUG:row_match_recognize.src.matcher.matcher:Deferred DEFINE condition passed for A at row 0
DEBUG:row_match_recognize.src.matcher.matcher:Validating deferred condition for C at row 1: event_type = 'end' AND C.value > FIRST(A.value)
DEBUG:row_match_recognize.src.matcher.row_context:Built indices: 3 row mappings, 0 subset mappings
DEBUG:row_match_recognize.src.matcher.condition_evaluator:Universal pattern variable 'event_type' resolved to: end
DEBUG:row_match_recognize.src.matcher.condition_evaluator:[DEBUG] COMPARE: left=end (<class 'str'>), op=Eq
DEBUG:row_match_recognize.src.matcher.condition_evaluator:[DEBUG] COMPARE: left=end (<class 'str'>), right=end (<class 'str'>)
DEBUG:row_match_recognize.src.matcher.condition_evaluator:[DEBUG] COMPARE RESULT: end Eq end = True (evaluating for var=C)
DEBUG:row_match_recognize.src.matcher.condition_evaluator:[DEBUG] _get_v

DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: True
DEBUG: Using backtracking matcher for complex pattern
DEBUG: Starting backtracking search with 3 rows, max_iterations=5000
DEBUG: Before sorting, 3 successors found
DEBUG: Successor 0: vars=['C'], last_var=C, row=1
DEBUG: Successor 1: vars=['A'], last_var=A, row=1
DEBUG: Successor 2: vars=['B'], last_var=B, row=1
DEBUG: Using individual variable priority 999 for non-PERMUTE
DEBUG: Using individual variable priority 999 for non-PERMUTE
DEBUG: Using individual variable priority 999 for non-PERMUTE
DEBUG: After sorting, successors order:
DEBUG: Using individual variable priority 999 for non-PERMUTE
DEBUG: Successor 0: vars=['A'], last_var=A, priority=999
DEBUG: Using individual variable priority 999 for non-PERMUTE
DEBUG: Successor 1: vars=['B'], last_var=B, priority=999
DEBUG: Using individual variable priority 999 for non-PERMUTE
DEBUG: Successor 2: vars=['C

DEBUG:row_match_recognize.src.matcher.production_aggregates:Pattern match result for 'MATCH_NUMBER()': <re.Match object; span=(0, 13), match='MATCH_NUMBER('>
DEBUG:row_match_recognize.src.matcher.production_aggregates:Matched function: MATCH_NUMBER, semantics_prefix: None
DEBUG:row_match_recognize.src.matcher.production_aggregates:Function MATCH_NUMBER not in standard aggregates, using original evaluator
INFO:row_match_recognize.src.matcher.production_aggregates:ENHANCED_EVAL_DEBUG: Falling back to original evaluator for: MATCH_NUMBER()
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Evaluating expression: MATCH_NUMBER() with FINAL semantics
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Context variables: {'A': [0], 'B': [1], 'C': [2]}
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Number of rows: 12
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Current index: 2
INFO:row_match_recognize.src.matcher.production_aggregates:ENHANCED_EVAL_DEBUG: Original

   seq  step pattern_var  match_num  start_value  middle_value  end_value
0    1     1           C          1          100           200        300
1    3     1           B          1          175           375        275




In [19]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Create test data with different permutation patterns
data = [
    # Sequence 1: Has A-B-C pattern
    {"id": 1, "seq": 1, "step": 1, "event_type": "start", "value": 100},  # A
    {"id": 2, "seq": 1, "step": 2, "event_type": "middle", "value": 200}, # B
    {"id": 3, "seq": 1, "step": 3, "event_type": "end", "value": 300},    # C
    
    # Sequence 2: Has B-A-C pattern
    {"id": 4, "seq": 2, "step": 1, "event_type": "middle", "value": 250}, # B
    {"id": 5, "seq": 2, "step": 2, "event_type": "start", "value": 150},  # A
    {"id": 6, "seq": 2, "step": 3, "event_type": "end", "value": 350},    # C
    
    # Sequence 3: Has A-C-B pattern
    {"id": 7, "seq": 3, "step": 1, "event_type": "start", "value": 175},  # A
    {"id": 8, "seq": 3, "step": 2, "event_type": "end", "value": 275},    # C
    {"id": 9, "seq": 3, "step": 3, "event_type": "middle", "value": 375}, # B
    
    # Sequence 4: Has C-B-A pattern
    {"id": 10, "seq": 4, "step": 1, "event_type": "end", "value": 225},   # C
    {"id": 11, "seq": 4, "step": 2, "event_type": "middle", "value": 325}, # B
    {"id": 12, "seq": 4, "step": 3, "event_type": "start", "value": 425},  # A
]

df = pd.DataFrame(data)

print("Testing PERMUTE Patterns\n")
# Test 6: PERMUTE with Complex Conditions
query_permute_complex = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        A.value AS start_value,
        B.value AS middle_value,
        C.value AS end_value
    ONE ROW PER MATCH
    PATTERN (PERMUTE(A, B, C))
    DEFINE 
        A AS event_type = 'start' AND A.value < NEXT(A.value),
        B AS event_type = 'middle' AND B.value > PREV(B.value),
        C AS event_type = 'end' AND C.value > FIRST(A.value)
);
"""

print("Test 6: PERMUTE with Complex Conditions - Testing complex pattern definitions")
output_df = match_recognize(query_permute_complex, df)
print(output_df)
print("\n")

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.op2 MATCH_RECOGNIZE( PARTITION BY seq ORDER BY step MEASURES CLASSIFIER() AS pattern_var, MATCH_NUMBER() AS match_num, A.value AS start_value, B.value AS middle_value, C.value AS end_value ONE ROW PER MATCH PATTERN (PERMUTE(A, B, C)) DEFINE A AS event_type = 'start' AND A.value < NEXT(A.value), B AS event_type = 'middle' AND B.value > PREV(B.value), C AS event_type = 'end' AND C.value > FIRST(A.value) );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['seq'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem

Testing PERMUTE Patterns

Test 6: PERMUTE with Complex Conditions - Testing complex pattern definitions
Pattern value: 'PERMUTE(A, B, C)'
Pattern value: 'PERMUTE(A, B, C)'
DEBUG: DFA metadata keys: ['has_permute', 'permute_variables', 'permute_structure', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: True
DEBUG: has_alternations: False
DEBUG: No alternation_combinations in DFA metadata
DEBUG: Parsed alternation_order: {}
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: True
DEBUG: Using backtracking matcher for complex pattern
DEBUG: Starting backtracking search with

INFO:row_match_recognize.src.matcher.production_aggregates:ENHANCED_EVAL_DEBUG: Original evaluator returned: C (type: <class 'str'>)
DEBUG:row_match_recognize.src.executor.match_recognize:Setting pattern_var to C from evaluator
DEBUG:row_match_recognize.src.matcher.production_aggregates:Enhanced evaluate called with expr: 'MATCH_NUMBER()', semantics: 'FINAL'
DEBUG:row_match_recognize.src.matcher.production_aggregates:Pattern match result for 'MATCH_NUMBER()': <re.Match object; span=(0, 13), match='MATCH_NUMBER('>
DEBUG:row_match_recognize.src.matcher.production_aggregates:Matched function: MATCH_NUMBER, semantics_prefix: None
DEBUG:row_match_recognize.src.matcher.production_aggregates:Function MATCH_NUMBER not in standard aggregates, using original evaluator
INFO:row_match_recognize.src.matcher.production_aggregates:ENHANCED_EVAL_DEBUG: Falling back to original evaluator for: MATCH_NUMBER()
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Evaluating expression: MATCH_NUMBER() wi

   seq  step pattern_var  match_num  start_value  middle_value  end_value
0    1     1           C          1          100           200        300
1    3     1           B          1          175           375        275




In [20]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Create test data for PERMUTE with subset variables
data = [
    # Sequence 1: Has A-B-C pattern
    {"id": 1, "seq": 1, "step": 1, "event_type": "start", "value": 100},  # A
    {"id": 2, "seq": 1, "step": 2, "event_type": "middle", "value": 200}, # B
    {"id": 3, "seq": 1, "step": 3, "event_type": "end", "value": 300},    # C
    
    # Sequence 2: Has B-A-C pattern
    {"id": 4, "seq": 2, "step": 1, "event_type": "middle", "value": 250}, # B
    {"id": 5, "seq": 2, "step": 2, "event_type": "start", "value": 150},  # A
    {"id": 6, "seq": 2, "step": 3, "event_type": "end", "value": 350},    # C
    
    # Sequence 3: Has A-C-B pattern
    {"id": 7, "seq": 3, "step": 1, "event_type": "start", "value": 175},  # A
    {"id": 8, "seq": 3, "step": 2, "event_type": "end", "value": 275},    # C
    {"id": 9, "seq": 3, "step": 3, "event_type": "middle", "value": 375}, # B
    
    # Sequence 4: Has C-B-A pattern
    {"id": 10, "seq": 4, "step": 1, "event_type": "end", "value": 225},   # C
    {"id": 11, "seq": 4, "step": 2, "event_type": "middle", "value": 325}, # B
    {"id": 12, "seq": 4, "step": 3, "event_type": "start", "value": 425},  # A
]

df = pd.DataFrame(data)

print("Testing PERMUTE with Subset Variables - Trino Compatibility\n")


# Test 7: PERMUTE with Edge Cases
query_permute_edge_cases = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        A.value AS a_value,
        LAST(B.value) AS last_b_value,
        FIRST(C.value) AS first_c_value
    ALL ROWS PER MATCH
    PATTERN (PERMUTE(A, B?, C?))
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

print("Test 7: PERMUTE with Edge Cases - Testing optional elements")
output_df = match_recognize(query_permute_edge_cases, df)
print(output_df)

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.op2 MATCH_RECOGNIZE( PARTITION BY seq ORDER BY step MEASURES CLASSIFIER() AS pattern_var, MATCH_NUMBER() AS match_num, A.value AS a_value, LAST(B.value) AS last_b_value, FIRST(C.value) AS first_c_value ALL ROWS PER MATCH PATTERN (PERMUTE(A, B?, C?)) DEFINE A AS event_type = 'start', B AS event_type = 'middle', C AS event_type = 'end' );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['seq'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='step', ordering='ASC', nulls_ordering=None)])
DEBUG:src.pars

Testing PERMUTE with Subset Variables - Trino Compatibility

Test 7: PERMUTE with Edge Cases - Testing optional elements
Pattern value: 'PERMUTE(A, B?, C?)'
Pattern value: 'PERMUTE(A, B?, C?)'
DEBUG: _process_permute called with variables: ['A', 'B?', 'C?']
DEBUG: Token metadata: {'variables': ['A', 'B?', 'C?'], 'variable_count': 3, 'original': 'PERMUTE(A, B?, C?)', 'has_nested': False, 'has_alternations': False, 'nested': False, 'depth': 0}
DEBUG: Token type: PatternTokenType.PERMUTE


DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='A', target=6, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='B', target=8, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='C', target=10, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='A', target=13, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='B', target=15, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='C', target=17, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='A', target=20, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='C', target=22, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='B', target=24, priority=0
DEBUG:row_match_recognize.src.matcher.automata:Added transition: variable='B', target=27, pri

DEBUG: DFA metadata keys: ['has_permute', 'permute_variables', 'permute_structure', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: True
DEBUG: has_alternations: False
DEBUG: No alternation_combinations in DFA metadata
DEBUG: Parsed alternation_order: {}
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: False
DEBUG: has_complex_back_references: False
DEBUG: Processing row 0 with value 100
DEBUG: Found 1 valid transitions: ['A']
DEBUG: Categorized transitions: {'accepting': [], 'prerequisite': [], 'simple': [('A', 2, False)], 'dependent': []}
DEBUG: Processing category '

DEBUG:row_match_recognize.src.matcher.matcher:    Condition passed for A
DEBUG:row_match_recognize.src.matcher.matcher:    DEBUG: condition result=True, type=<class 'bool'>
DEBUG:row_match_recognize.src.matcher.matcher:  [DEBUG] Clearing context.current_var (was A)
DEBUG:row_match_recognize.src.matcher.matcher:  Transition A: accepting=False, has_back_ref=False, is_prerequisite=False
DEBUG:row_match_recognize.src.matcher.matcher:  Assigned row 1 to variable A
DEBUG:row_match_recognize.src.matcher.matcher:Testing row 2, data: {'id': 6, 'seq': 2, 'step': 3, 'event_type': 'end', 'value': 350}
DEBUG:row_match_recognize.src.matcher.matcher:  Current var_assignments: {'B': [0], 'A': [1]}
DEBUG:row_match_recognize.src.matcher.matcher:  Evaluating condition for var: C
DEBUG:row_match_recognize.src.matcher.matcher:  [DEBUG] Set context.current_var = C
DEBUG:row_match_recognize.src.matcher.matcher:    DEBUG: Calling condition function with row={'id': 6, 'seq': 2, 'step': 3, 'event_type': 'end', 

DEBUG: Found 1 valid transitions: ['A']
DEBUG: Categorized transitions: {'accepting': [], 'prerequisite': [], 'simple': [('A', 9, False)], 'dependent': []}
DEBUG: Processing category 'simple' with 1 transitions
DEBUG: Selected simple transition: A -> state 9 (alternation priority: N/A)
DEBUG: Processing row 2 with value 350
DEBUG: Found 1 valid transitions: ['C']
DEBUG: Categorized transitions: {'accepting': [('C', 12, False)], 'prerequisite': [], 'simple': [], 'dependent': []}
DEBUG: Processing category 'accepting' with 1 transitions
DEBUG: Selected accepting transition: C -> state 12 (alternation priority: N/A)
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: False
DEBUG: has_complex_back_references: False
DEBUG: Processing row 0 with value 175


DEBUG:row_match_recognize.src.matcher.condition_evaluator:Universal pattern variable 'event_type' resolved to: start
DEBUG:row_match_recognize.src.matcher.condition_evaluator:[DEBUG] COMPARE: left=start (<class 'str'>), op=Eq
DEBUG:row_match_recognize.src.matcher.condition_evaluator:[DEBUG] COMPARE: left=start (<class 'str'>), right=end (<class 'str'>)
DEBUG:row_match_recognize.src.matcher.condition_evaluator:[DEBUG] COMPARE RESULT: start Eq end = False (evaluating for var=C)
DEBUG:row_match_recognize.src.matcher.condition_evaluator:Universal pattern variable 'event_type' resolved to: start
DEBUG:row_match_recognize.src.matcher.condition_evaluator:[DEBUG] COMPARE: left=start (<class 'str'>), op=Eq
DEBUG:row_match_recognize.src.matcher.condition_evaluator:[DEBUG] COMPARE: left=start (<class 'str'>), right=end (<class 'str'>)
DEBUG:row_match_recognize.src.matcher.condition_evaluator:[DEBUG] COMPARE RESULT: start Eq end = False (evaluating for var=C)
DEBUG:row_match_recognize.src.matcher.

DEBUG: Found 1 valid transitions: ['A']
DEBUG: Categorized transitions: {'accepting': [], 'prerequisite': [], 'simple': [('A', 2, False)], 'dependent': []}
DEBUG: Processing category 'simple' with 1 transitions
DEBUG: Selected simple transition: A -> state 2 (alternation priority: N/A)
DEBUG: Processing row 1 with value 275
DEBUG: Found 1 valid transitions: ['C']
DEBUG: Categorized transitions: {'accepting': [], 'prerequisite': [], 'simple': [('C', 6, False)], 'dependent': []}
DEBUG: Processing category 'simple' with 1 transitions
DEBUG: Selected simple transition: C -> state 6 (alternation priority: N/A)
DEBUG: Processing row 2 with value 375
DEBUG: Found 1 valid transitions: ['B']
DEBUG: Categorized transitions: {'accepting': [('B', 11, False)], 'prerequisite': [], 'simple': [], 'dependent': []}
DEBUG: Processing category 'accepting' with 1 transitions
DEBUG: Selected accepting transition: B -> state 11 (alternation priority: N/A)
DEBUG: _find_single_match called with start_idx=0
DEB

DEBUG:row_match_recognize.src.matcher.matcher:Row 1: Measure 'pattern_var' = 'CLASSIFIER()' using FINAL semantics
DEBUG:row_match_recognize.src.matcher.production_aggregates:Enhanced evaluate called with expr: 'CLASSIFIER()', semantics: 'FINAL'
DEBUG:row_match_recognize.src.matcher.production_aggregates:Pattern match result for 'CLASSIFIER()': <re.Match object; span=(0, 11), match='CLASSIFIER('>
DEBUG:row_match_recognize.src.matcher.production_aggregates:Matched function: CLASSIFIER, semantics_prefix: None
DEBUG:row_match_recognize.src.matcher.production_aggregates:Function CLASSIFIER not in standard aggregates, using original evaluator
INFO:row_match_recognize.src.matcher.production_aggregates:ENHANCED_EVAL_DEBUG: Falling back to original evaluator for: CLASSIFIER()
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Evaluating expression: CLASSIFIER() with FINAL semantics
DEBUG:row_match_recognize.src.matcher.measure_evaluator:Context variables: {'C': [0], 'B': [1], 'A': [2]}
DEB

    seq  step pattern_var  match_num a_value last_b_value first_c_value  id  \
0     1     1           A          1     100         None          None   1   
1     1     2           B          1     100          200          None   2   
2     1     3           C          1     100          200           300   3   
3     2     1           B          1    None          250          None   4   
4     2     2           A          1     150          250          None   5   
5     2     3           C          1     150          250           350   6   
6     3     1           A          1     175         None          None   7   
7     3     2           C          1     175         None           275   8   
8     3     3           B          1     175          375           275   9   
9     4     1           C          1    None         None           225  10   
10    4     2           B          1    None          325           225  11   
11    4     3           A          1     425        

# Example 3


In [21]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Validation query with ALL ROWS PER MATCH
query = """
    SELECT * FROM memory.default.employees MATCH_RECOGNIZE (
        PARTITION BY department, region
        ORDER BY hire_date
        MEASURES 
            salary AS current_salary,
            RUNNING SUM(salary) AS running_sum,
            MATCH_NUMBER() AS match_num
        ALL ROWS PER MATCH
        PATTERN (A+)
        DEFINE A AS salary > 1000
    );
    """
    
data = [
    {"id": 1, "name": "Alice",   "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
    {"id": 2, "name": "Bob",     "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
    {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
    {"id": 4, "name": "Diana",   "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
]
    
output_df = match_recognize(query, pd.DataFrame(data))
print("Match Recognize Output:")
print(output_df)


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.employees MATCH_RECOGNIZE ( PARTITION BY department, region ORDER BY hire_date MEASURES salary AS current_salary, RUNNING SUM(salary) AS running_sum, MATCH_NUMBER() AS match_num ALL ROWS PER MATCH PATTERN (A+) DEFINE A AS salary > 1000 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['department', 'region'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='hire_date', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Meas

Pattern value: 'A+'
Pattern value: 'A+'
DEBUG: DFA metadata keys: ['pattern_variables', 'variables_with_quantifiers', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: False
DEBUG: has_alternations: False
DEBUG: No alternation_combinations in DFA metadata
DEBUG: Parsed alternation_order: {}
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: False
DEBUG: has_complex_back_references: False
DEBUG: Processing row 0 with value N/A
DEBUG: Found 1 valid transitions: ['A']
DEBUG: Categorized transitions: {'accepting': [('A', 1, False)], 'prerequisite': [], 'simple': [], 'dependent

In [22]:
import pandas as pd
from src.executor.match_recognize import match_recognize
# Use an absolute import for match_recognize.

query = """
    SELECT * FROM memory.default.employees MATCH_RECOGNIZE (
        PARTITION BY department, region
        ORDER BY hire_date
        MEASURES salary AS avg_salary
        PATTERN (A+)
        DEFINE A AS salary > 1000
    );
    """
    
data = [
        {"id": 1, "name": "Alice",   "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
        {"id": 2, "name": "Bob",     "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
        {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
        {"id": 4, "name": "Diana",   "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
    ]
    
output_df = match_recognize(query, pd.DataFrame(data))
print("Match Recognize Output:")
print(output_df)


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.employees MATCH_RECOGNIZE ( PARTITION BY department, region ORDER BY hire_date MEASURES salary AS avg_salary PATTERN (A+) DEFINE A AS salary > 1000 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['department', 'region'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='hire_date', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='salary', alias='avg_salary', metadata={'semantics': 'FINAL', 'explicit_s

Pattern value: 'A+'
Pattern value: 'A+'
DEBUG: DFA metadata keys: ['pattern_variables', 'variables_with_quantifiers', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: False
DEBUG: has_alternations: False
DEBUG: No alternation_combinations in DFA metadata
DEBUG: Parsed alternation_order: {}
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: False
DEBUG: has_complex_back_references: False
DEBUG: Processing row 0 with value N/A
DEBUG: Found 1 valid transitions: ['A']
DEBUG: Categorized transitions: {'accepting': [('A', 1, False)], 'prerequisite': [], 'simple': [], 'dependent

In [23]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Validation query with ALL ROWS PER MATCH
query = """
    SELECT * FROM memory.default.employees MATCH_RECOGNIZE (
        PARTITION BY department, region
        ORDER BY hire_date
        MEASURES 
            salary AS current_salary,
            RUNNING SUM(salary) AS running_sum,
            MATCH_NUMBER() AS match_num
        ALL ROWS PER MATCH
        PATTERN (A*)
        DEFINE A AS salary > 1000
    );
    """
    
data = [
    {"id": 1, "name": "Alice",   "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
    {"id": 2, "name": "Bob",     "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
    {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
    {"id": 4, "name": "Diana",   "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
]
    
output_df = match_recognize(query, pd.DataFrame(data))
print("Match Recognize Output:")
print(output_df)


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.employees MATCH_RECOGNIZE ( PARTITION BY department, region ORDER BY hire_date MEASURES salary AS current_salary, RUNNING SUM(salary) AS running_sum, MATCH_NUMBER() AS match_num ALL ROWS PER MATCH PATTERN (A*) DEFINE A AS salary > 1000 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['department', 'region'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='hire_date', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Meas

Pattern value: 'A*'
Pattern value: 'A*'
DEBUG: DFA metadata keys: ['allows_empty', 'pattern_variables', 'variables_with_quantifiers', 'allows_empty_match', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: False
DEBUG: has_alternations: False
DEBUG: No alternation_combinations in DFA metadata
DEBUG: Parsed alternation_order: {}
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: False
DEBUG: has_complex_back_references: False
DEBUG: Processing row 0 with value N/A
DEBUG: Found 1 valid transitions: ['A']
DEBUG: Categorized transitions: {'accepting': [('A', 0, False)], 'prere

In [25]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Validation query with ALL ROWS PER MATCH
query = """
SELECT *
FROM memory.default.employees 
MATCH_RECOGNIZE (
  PARTITION BY department, region
  ORDER BY hire_date
  MEASURES 
    A.salary AS starting_salary,
    LAST(C.salary) AS ending_salary,
    MATCH_NUMBER() AS match_num
  ONE ROW PER MATCH
  AFTER MATCH SKIP PAST LAST ROW
  PATTERN (A B+ C+)
  DEFINE 
    A AS salary > 1000,
    B AS salary < 1000,
    C AS salary > 1000
);


    """
    
data = [
    {"id": 1, "name": "Alice",   "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
    {"id": 2, "name": "Bob",     "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
    {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
    {"id": 4, "name": "Diana",   "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
]
    
output_df = match_recognize(query, pd.DataFrame(data))
print("Match Recognize Output:")
print(output_df)


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.employees MATCH_RECOGNIZE ( PARTITION BY department, region ORDER BY hire_date MEASURES A.salary AS starting_salary, LAST(C.salary) AS ending_salary, MATCH_NUMBER() AS match_num ONE ROW PER MATCH AFTER MATCH SKIP PAST LAST ROW PATTERN (A B+ C+) DEFINE A AS salary > 1000, B AS salary < 1000, C AS salary > 1000 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['department', 'region'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='hire_date', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.

Pattern value: 'A B+ C+'
Pattern value: 'A B+ C+'
Creating transition for variable 'A' with condition: 'salary > 1000'
Creating transition for variable 'B' with condition: 'salary < 1000'
Creating transition for variable 'C' with condition: 'salary > 1000'
Initialized matcher with excluded variables: set()
Find matches with all_rows=False, show_empty=True, include_unmatched=False
Starting match at index 0, state: State 0 (Non-accept, Vars: A)
Testing row 0, data: {'id': 1, 'name': 'Alice', 'department': 'Sales', 'region': 'West', 'hire_date': '2021-01-01', 'salary': 1200}
  Evaluating condition for var: A
    Condition passed for A
  Assigned row 0 to variable A
Testing row 1, data: {'id': 2, 'name': 'Bob', 'department': 'Sales', 'region': 'West', 'hire_date': '2021-01-02', 'salary': 1300}
  Evaluating condition for var: B
    Condition failed for B
No valid transition from state 1 at row 1
No match found starting at index 0
Starting match at index 1, state: State 0 (Non-accept, Vars: 

In [24]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Example query with comprehensive CLASSIFIER usage
query = """
SELECT * FROM memory.default.employees  MATCH_RECOGNIZE(
    PARTITION BY department, region
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        salary AS current_salary,
        RUNNING SUM(salary) AS running_sum
    ALL ROWS PER MATCH
    PATTERN (A C* {- B+ -} C+)
    DEFINE 
        A AS salary > 1000,
        B AS salary < 1000,
        C AS salary > 1000
);

"""

data = [
    {"id": 1, "name": "Alice", "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
    {"id": 2, "name": "Bob",   "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
    {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
    {"id": 4, "name": "Diana", "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
]


output_df = match_recognize(query, pd.DataFrame(data))
print("Match Recognize Output:")
print(output_df)

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.employees MATCH_RECOGNIZE( PARTITION BY department, region ORDER BY hire_date MEASURES CLASSIFIER() AS pattern_var, salary AS current_salary, RUNNING SUM(salary) AS running_sum ALL ROWS PER MATCH PATTERN (A C* {- B+ -} C+) DEFINE A AS salary > 1000, B AS salary < 1000, C AS salary > 1000 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['department', 'region'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='hire_date', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extrac

Pattern value: 'A C* {- B+ -} C+'
Pattern value: 'A C* {- B+ -} C+'
DEBUG: DFA metadata keys: ['excluded_variables', 'allows_empty', 'pattern_variables', 'variables_with_quantifiers', 'allows_empty_match', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: False
DEBUG: has_alternations: False
DEBUG: No alternation_combinations in DFA metadata
DEBUG: Parsed alternation_order: {}
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: False
DEBUG: has_complex_back_references: False
DEBUG: Processing row 0 with value N/A
DEBUG: Found 1 valid transitions: ['A']
DEBUG: Categorized tr

In [25]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Example query with comprehensive CLASSIFIER usage
query = """
SELECT * FROM memory.default.employees  MATCH_RECOGNIZE(
    PARTITION BY department, region
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        salary AS current_salary,
        RUNNING SUM(salary) AS running_sum
    ALL ROWS PER MATCH
    PATTERN (A {- B+ -} C+)
    DEFINE 
        A AS salary > 1000,
        B AS salary < 1000,
        C AS salary > 1000
);

"""

data = [
    {"id": 1, "name": "Alice", "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
    {"id": 2, "name": "Bob",   "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
    {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
    {"id": 4, "name": "Diana", "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
]


output_df = match_recognize(query, pd.DataFrame(data))
print("Match Recognize Output:")
print(output_df)

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.employees MATCH_RECOGNIZE( PARTITION BY department, region ORDER BY hire_date MEASURES CLASSIFIER() AS pattern_var, salary AS current_salary, RUNNING SUM(salary) AS running_sum ALL ROWS PER MATCH PATTERN (A {- B+ -} C+) DEFINE A AS salary > 1000, B AS salary < 1000, C AS salary > 1000 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['department', 'region'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='hire_date', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor

Pattern value: 'A {- B+ -} C+'
Pattern value: 'A {- B+ -} C+'
DEBUG: DFA metadata keys: ['excluded_variables', 'pattern_variables', 'variables_with_quantifiers', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: False
DEBUG: has_alternations: False
DEBUG: No alternation_combinations in DFA metadata
DEBUG: Parsed alternation_order: {}
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: False
DEBUG: has_complex_back_references: False
DEBUG: Processing row 0 with value N/A
DEBUG: Found 1 valid transitions: ['A']
DEBUG: Categorized transitions: {'accepting': [], 'prerequisite':

In [26]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Example query with comprehensive CLASSIFIER usage
query = """
SELECT * FROM  memory.default.employees MATCH_RECOGNIZE(
    PARTITION BY department, region
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        CLASSIFIER(A) AS is_a_var,
        CLASSIFIER(C) AS is_c_var,
        salary AS current_salary,
        RUNNING SUM(salary) AS running_sum
    ONE ROW PER MATCH
    PATTERN (A {- B+ -} C+)
    DEFINE 
        A AS salary > 1000,
        B AS salary < 1000,
        C AS salary > 1000
);
"""

data = [
    {"id": 1, "name": "Alice", "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
    {"id": 2, "name": "Bob",   "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
    {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
    {"id": 4, "name": "Diana", "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
]


output_df = match_recognize(query, pd.DataFrame(data))
print("Match Recognize Output:")
print(output_df)

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.employees MATCH_RECOGNIZE( PARTITION BY department, region ORDER BY hire_date MEASURES CLASSIFIER() AS pattern_var, CLASSIFIER(A) AS is_a_var, CLASSIFIER(C) AS is_c_var, salary AS current_salary, RUNNING SUM(salary) AS running_sum ONE ROW PER MATCH PATTERN (A {- B+ -} C+) DEFINE A AS salary > 1000, B AS salary < 1000, C AS salary > 1000 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['department', 'region'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='hire_date', ordering='ASC', nulls_orderi

Pattern value: 'A {- B+ -} C+'
Pattern value: 'A {- B+ -} C+'
DEBUG: DFA metadata keys: ['excluded_variables', 'pattern_variables', 'variables_with_quantifiers', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: False
DEBUG: has_alternations: False
DEBUG: No alternation_combinations in DFA metadata
DEBUG: Parsed alternation_order: {}
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: False
DEBUG: has_complex_back_references: False
DEBUG: Processing row 0 with value N/A
DEBUG: Found 1 valid transitions: ['A']
DEBUG: Categorized transitions: {'accepting': [], 'prerequisite':

In [27]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Example query with comprehensive CLASSIFIER usage
query = """
SELECT * FROM memory.default.employees MATCH_RECOGNIZE(
    PARTITION BY department
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num
    ONE ROW PER MATCH
    PATTERN (^A+)
    DEFINE 
        A AS salary > 1000
);
"""

data = [
    {"id": 1, "name": "Alice", "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
    {"id": 2, "name": "Bob",   "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
    {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
    {"id": 4, "name": "Diana", "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
]


output_df = match_recognize(query, pd.DataFrame(data))
print("Match Recognize Output:")
print(output_df)

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.employees MATCH_RECOGNIZE( PARTITION BY department ORDER BY hire_date MEASURES CLASSIFIER() AS pattern_var, MATCH_NUMBER() AS match_num ONE ROW PER MATCH PATTERN (^A+) DEFINE A AS salary > 1000 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['department'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='hire_date', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='CLASSIFIER()', alias='pattern_var', 

Pattern value: '^A+'
Pattern value: '^A+'
DEBUG: DFA metadata keys: ['has_start_anchor', 'pattern_variables', 'variables_with_quantifiers', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: False
DEBUG: has_alternations: False
DEBUG: No alternation_combinations in DFA metadata
DEBUG: Parsed alternation_order: {}
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: False
DEBUG: has_complex_back_references: False
DEBUG: Processing row 0 with value N/A
DEBUG: Found 1 valid transitions: ['A']
DEBUG: Categorized transitions: {'accepting': [('A', 1, False)], 'prerequisite': [], 's

# Example 4


In [28]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Create test data with different departments to test partition behavior
data = [
    # Sales department - First row has high salary
    {"id": 1, "name": "Alice", "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
    {"id": 2, "name": "Bob",   "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
    {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
    {"id": 4, "name": "Diana", "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
    
    # Marketing department - Last row has high salary
    {"id": 5, "name": "Eve", "department": "Marketing", "region": "East", "hire_date": "2021-01-01", "salary": 900},
    {"id": 6, "name": "Frank", "department": "Marketing", "region": "East", "hire_date": "2021-01-02", "salary": 950},
    {"id": 7, "name": "Grace", "department": "Marketing", "region": "East", "hire_date": "2021-01-03", "salary": 980},
    {"id": 8, "name": "Henry", "department": "Marketing", "region": "East", "hire_date": "2021-01-04", "salary": 1200},
    
    # IT department - All rows have high salary
    {"id": 9, "name": "Ivy", "department": "IT", "region": "North", "hire_date": "2021-01-01", "salary": 1500},
    {"id": 10, "name": "Jack", "department": "IT", "region": "North", "hire_date": "2021-01-02", "salary": 1600},
    {"id": 11, "name": "Kate", "department": "IT", "region": "North", "hire_date": "2021-01-03", "salary": 1700},
    {"id": 12, "name": "Leo", "department": "IT", "region": "North", "hire_date": "2021-01-04", "salary": 1800},
    
    # HR department - No rows have high salary
    {"id": 13, "name": "Mike", "department": "HR", "region": "South", "hire_date": "2021-01-01", "salary": 950},
    {"id": 14, "name": "Nina", "department": "HR", "region": "South", "hire_date": "2021-01-02", "salary": 980},
    {"id": 15, "name": "Oscar", "department": "HR", "region": "South", "hire_date": "2021-01-03", "salary": 990},
    {"id": 16, "name": "Pam", "department": "HR", "region": "South", "hire_date": "2021-01-04", "salary": 995},
]

df = pd.DataFrame(data)

print("Testing Pattern Anchors\n")

# Test 1: Start anchor (^) - Should match patterns starting at the beginning of a partition
query_start_anchor = """
SELECT * FROM memory.default.employee_data MATCH_RECOGNIZE(
    PARTITION BY department
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num
    ONE ROW PER MATCH
    PATTERN (^A+)
    DEFINE 
        A AS salary > 1000
);
"""

print("Test 1: Start Anchor (^) - Should only match departments where first employee has salary > 1000")
output_df = match_recognize(query_start_anchor, df)
print(output_df)
print("\n")


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.employee_data MATCH_RECOGNIZE( PARTITION BY department ORDER BY hire_date MEASURES CLASSIFIER() AS pattern_var, MATCH_NUMBER() AS match_num ONE ROW PER MATCH PATTERN (^A+) DEFINE A AS salary > 1000 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['department'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='hire_date', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='CLASSIFIER()', alias='pattern_va

Testing Pattern Anchors

Test 1: Start Anchor (^) - Should only match departments where first employee has salary > 1000
Pattern value: '^A+'
Pattern value: '^A+'
DEBUG: DFA metadata keys: ['has_start_anchor', 'pattern_variables', 'variables_with_quantifiers', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: False
DEBUG: has_alternations: False
DEBUG: No alternation_combinations in DFA metadata
DEBUG: Parsed alternation_order: {}
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: False
DEBUG: has_complex_back_references: False
DEBUG: Processing row 0 with value N/A
DEBUG:

In [35]:

# Test 2: End anchor ($) - Should match patterns ending at the end of a partition
query_end_anchor = """
SELECT * FROM memory.default.employee_data MATCH_RECOGNIZE(
    PARTITION BY department
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num
    ONE ROW PER MATCH
    PATTERN (A+$)
    DEFINE 
        A AS salary > 1000
);
"""

print("Test 2: End Anchor ($) - Should only match departments where last employee has salary > 1000")
output_df = match_recognize(query_end_anchor, df)
print(output_df)
print("\n")



DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.employee_data MATCH_RECOGNIZE( PARTITION BY department ORDER BY hire_date MEASURES CLASSIFIER() AS pattern_var, MATCH_NUMBER() AS match_num ONE ROW PER MATCH PATTERN (A+$) DEFINE A AS salary > 1000 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['department'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='hire_date', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='CLASSIFIER()', alias='pattern_va

Test 2: End Anchor ($) - Should only match departments where last employee has salary > 1000
Pattern value: 'A+$'
Pattern value: 'A+$'
DEBUG: DFA metadata keys: ['has_end_anchor', 'pattern_variables', 'variables_with_quantifiers', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: False
DEBUG: has_alternations: False
DEBUG: No alternation_combinations in DFA metadata
DEBUG: Parsed alternation_order: {}
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: False
DEBUG: has_complex_back_references: False
DEBUG: Processing row 0 with value N/A
DEBUG: Found 1 valid transitions: ['

In [36]:
# Test 3: Both anchors (^$) - Should match patterns spanning the entire partition
query_both_anchors = """
SELECT * FROM memory.default.employee_data MATCH_RECOGNIZE(
    PARTITION BY department
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num
    ONE ROW PER MATCH
    PATTERN (^A+$)
    DEFINE 
        A AS salary > 1000
);
"""

print("Test 3: Both Anchors (^$) - Should only match departments where ALL employees have salary > 1000")
output_df = match_recognize(query_both_anchors, df)
print(output_df)
print("\n")


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.employee_data MATCH_RECOGNIZE( PARTITION BY department ORDER BY hire_date MEASURES CLASSIFIER() AS pattern_var, MATCH_NUMBER() AS match_num ONE ROW PER MATCH PATTERN (^A+$) DEFINE A AS salary > 1000 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['department'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='hire_date', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='CLASSIFIER()', alias='pattern_v

Test 3: Both Anchors (^$) - Should only match departments where ALL employees have salary > 1000
Pattern value: '^A+$'
Pattern value: '^A+$'
DEBUG: DFA metadata keys: ['has_start_anchor', 'has_end_anchor', 'spans_partition', 'pattern_variables', 'variables_with_quantifiers', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: False
DEBUG: has_alternations: False
DEBUG: No alternation_combinations in DFA metadata
DEBUG: Parsed alternation_order: {}
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: False
DEBUG: has_complex_back_references: False
DEBUG: Processing row 0 with v

In [37]:

# Test 4: Start anchor with ALL ROWS PER MATCH to see the actual matched rows
query_start_all_rows = """
SELECT * FROM memory.default.employee_data MATCH_RECOGNIZE(
    PARTITION BY department
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num
    ALL ROWS PER MATCH
    PATTERN (^A+)
    DEFINE 
        A AS salary > 1000
);
"""

print("Test 4: Start Anchor (^) with ALL ROWS PER MATCH - Shows matched rows")
output_df = match_recognize(query_start_all_rows, df)
print(output_df)


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.employee_data MATCH_RECOGNIZE( PARTITION BY department ORDER BY hire_date MEASURES CLASSIFIER() AS pattern_var, MATCH_NUMBER() AS match_num ALL ROWS PER MATCH PATTERN (^A+) DEFINE A AS salary > 1000 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['department'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='hire_date', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='CLASSIFIER()', alias='pattern_v

Test 4: Start Anchor (^) with ALL ROWS PER MATCH - Shows matched rows
Pattern value: '^A+'
Pattern value: '^A+'
DEBUG: DFA metadata keys: ['has_start_anchor', 'pattern_variables', 'variables_with_quantifiers', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: False
DEBUG: has_alternations: False
DEBUG: No alternation_combinations in DFA metadata
DEBUG: Parsed alternation_order: {}
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: False
DEBUG: has_complex_back_references: False
DEBUG: Processing row 0 with value N/A
DEBUG: Found 1 valid transitions: ['A']
DEBUG: Categorize

In [38]:
# Test PERMUTE functionality
query_permute = """
SELECT * FROM memory.default.employee_data MATCH_RECOGNIZE(
    PARTITION BY department
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num
    ONE ROW PER MATCH
    PATTERN (PERMUTE(A, B))
    DEFINE 
        A AS salary > 1200,
        B AS salary < 1000
);
"""

print("Test PERMUTE - Should match both orderings of A and B")
output_df = match_recognize(query_permute, df)
print(output_df)
print("\n")


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.employee_data MATCH_RECOGNIZE( PARTITION BY department ORDER BY hire_date MEASURES CLASSIFIER() AS pattern_var, MATCH_NUMBER() AS match_num ONE ROW PER MATCH PATTERN (PERMUTE(A, B)) DEFINE A AS salary > 1200, B AS salary < 1000 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['department'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='hire_date', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='CL

Test PERMUTE - Should match both orderings of A and B
Pattern value: 'PERMUTE(A, B)'
Pattern value: 'PERMUTE(A, B)'
DEBUG: DFA metadata keys: ['has_permute', 'permute_variables', 'permute_structure', 'original_nfa_states', 'original_nfa_transitions', 'construction_method', 'exponential_protection', 'optimization_features', 'build_stats', 'dfa_construction_time', 'cache_hit_rate', 'construction_efficiency', 'optimized', 'optimization_time', 'original_state_count', 'optimized_state_count', 'original_transition_count', 'optimized_transition_count', 'optimization_savings']
DEBUG: has_permute: True
DEBUG: has_alternations: False
DEBUG: No alternation_combinations in DFA metadata
DEBUG: Parsed alternation_order: {}
DEBUG: _find_single_match called with start_idx=0
DEBUG: has_permute_alternations: False
DEBUG: _needs_backtracking returned: False
DEBUG: has_complex_back_references: False
DEBUG: Processing row 0 with value N/A
DEBUG: _find_single_match called with start_idx=1
DEBUG: has_permute

DEBUG:row_match_recognize.src.matcher.measure_evaluator:Current index: 2
INFO:row_match_recognize.src.matcher.production_aggregates:ENHANCED_EVAL_DEBUG: Original evaluator returned: 1 (type: <class 'int'>)
DEBUG:row_match_recognize.src.executor.match_recognize:Setting match_num to 1 from evaluator
DEBUG:row_match_recognize.src.executor.match_recognize:ONE ROW PER MATCH with SELECT * - PARTITION BY: ['department'], ORDER BY: ['hire_date'], MEASURES: ['pattern_var', 'match_num']
DEBUG:row_match_recognize.src.executor.match_recognize:SELECT * - SQL:2016 column ordering: ['department', 'hire_date', 'pattern_var', 'match_num']
DEBUG:row_match_recognize.src.executor.match_recognize:Available columns in result_df: ['salary', 'id', 'region', 'hire_date', 'department', 'name', 'pattern_var', 'match_num']
DEBUG:row_match_recognize.src.executor.match_recognize:Measures: ['pattern_var', 'match_num']
DEBUG:row_match_recognize.src.executor.match_recognize:Rows per match mode: RowsPerMatch.ONE_ROW
IN

  department   hire_date pattern_var  match_num
0      Sales  2021-01-02           B          1


